# Cs/Th analysis

In [ ]:
%matplotlib inline

import os
import textwrap
import math
#import h5py

import matplotlib        as mpl
import matplotlib.pyplot as plt
import numpy             as np
import tables            as tb

import invisible_cities.core.core_functions as coref
import invisible_cities.core.fit_functions  as fitf
import invisible_cities.reco.dst_functions  as dstf
import invisible_cities.reco.tbl_functions  as tbl
import invisible_cities.sierpe.blr          as blr

from   invisible_cities.icaro. hst_functions   import display_matrix
from   invisible_cities.database               import load_db

from   matplotlib                              import gridspec
from   matplotlib.patches                      import Ellipse
from   scipy.optimize                          import curve_fit
from   scipy.interpolate                       import interp1d

from   matplotlib.colors                       import SymLogNorm

# Formatting options
mpl.rcParams.update({'font.size': 14})
mpl.rcParams['image.cmap'] = 'Greys'
mpl.rcParams['patch.force_edgecolor'] = False
mpl.rcParams['patch.facecolor'] = 'gray'
hargs = {'histtype': 'stepfilled', 'edgecolor': 'black', 'facecolor': 'gray'}

# Directory to which figures will be stored
save_dir = "fig_10bar_all"
ftype = "pdf"

## Function definitions

In [ ]:
# Fit to 2 Gaussians
def gauss2(x, amp1, mu1, sigma1, amp2, mu2, sigma2):
    if sigma1 <= 0. or sigma2 <= 0:
        return np.inf
    return amp1/(2*np.pi)**.5/sigma1 * np.exp(-0.5*(x-mu1)**2./sigma1**2.) + amp2/(2*np.pi)**.5/sigma2 * np.exp(-0.5*(x-mu2)**2./sigma2**2.)

# Fit to Gaussian + exponential
def gaussexpo(x, amp, mu, sigma, const, mean, x0):

    if sigma <= 0.:
        return np.inf
    
    return amp/(2*np.pi)**(0.5)/sigma * np.exp(-0.5*(x-mu)**2./sigma**2) + const * np.exp(-(x-x0)/mean)

# (code from Gonzalo)
# Return resolution extrapolated to Qbb
def reso(values,epeak):
    _, mu, sigma = values
    r = 235. * sigma/mu
    return r, r * (epeak/2458)**0.5

# Return errors on resolution extrapolated to Qbb
def reso_errors(values,errors,epeak):
    _, mu, sigma = values
    _, err_mu, err_sigma = errors
    r = 235. * sigma/mu
    err_r = r*np.sqrt((err_sigma/sigma)**2 + (err_mu/mu)**2)
    return err_r, err_r * (epeak/2458)**0.5

# Returns text output for Gaussian fit function
def gausstext(values,epeak):
    return textwrap.dedent("""
        $\mu$ = {:.1f}
        $\sigma$ = {:.2f}
        R = {:.3}%
        R$_\\beta$$_\\beta$ = {:.3}%""".format(*values[1:], *reso(values,epeak)))

# Returns text output for Gaussian + exponential fit function
def gaussexpotext(values,epeak):
    return textwrap.dedent("""
        $\mu$ = {:.1f}
        $\sigma$ = {:.2f}
        R = {:.3}%
        R$_\\beta$$_\\beta$ = {:.3}%""".format(*values[1:3], *reso(values[0:3],epeak)))

# Returns text output for two-Gaussian fit function
def gauss2text(values,epeak1,epeak2):
    return textwrap.dedent("""
        $\mu_1$ = {:.1f}
        $\sigma_1$ = {:.2f}
        R$_1$ = {:.3}%
        R$_1$$_\\beta$$_\\beta$ = {:.3}%
        $\mu_2$ = {:.1f}
        $\sigma_2$ = {:.2f}
        R$_2$ = {:.3}%
        R$_2$$_\\beta$$_\\beta$ = {:.3}%""".format(*values[1:3], *reso(values[0:3],epeak1), *values[4:], *reso(values[3:],epeak2)))

# Modified hst_functions.display_matrix
def display_matrix_mod(x, y, z, mask=None, **kwargs):
    """
    Display the matrix z using the coordinates x and y as the bin centers.
    """
    nx = np.size(x)
    ny = np.size(y)

    dx = (np.max(x) - np.min(x)) / nx
    dy = (np.max(y) - np.min(y)) / ny

    x_binning = np.linspace(np.min(x) - dx, np.max(x) + dx, nx + 1)
    y_binning = np.linspace(np.min(y) - dy, np.max(y) + dy, ny + 1)

    x_ = np.repeat(x, ny)
    y_ = np.tile  (y, nx)
    z_ = z.flatten()

    if mask is None:
        mask = np.ones_like(z_, dtype=bool)
        
    h, x, y = np.histogram2d(-y_[mask], x_[mask], bins=[x_binning, y_binning], weights=z_[mask], **kwargs)
    return h, x, y

# A linear function mx + b
def flinear(x, m, b):
    return m*x + b

# A quadratic function mx**2 + b
def fquadratic(x, m, b):
    return m*x**2 + b

# A sqrt function a*sqrt(x)
def fsqrt(x, a, b):
    return a*x**0.5 + b

# A 2nd order polynomial
def poly2(x, a, b, c):
    return a*x**2 + b*x + c

# Fit function for energy resolution.
def fres(x, a, b):
    return a + b/x**0.5

-------------------------

# Save/load main event quantities

In [ ]:
# Overall energy calibration, runs 6342 - 6485, z-length correction 2.1e-4
# Fit parameters are: a = -1.1554510080731935e-10, b = 0.0031788260410015694, c = 0.8680414331255459
# Residuals (%): [-0.1261996   0.15144126 -0.13989668  0.15086734]

# Overall energy calibration, runs 6342 - 6485, z-length correction 3.1e-4
# Fit parameters are: a = -1.4192533822211684e-10, b = 0.003177595385823009, c = 0.8821095680401282
# Residuals (%): [-0.12865034  0.15381911 -0.14130596  0.1514282 ]

CAL_a = -1.4192533822211684e-10
CAL_b = 0.003177595385823009
CAL_c = 0.8821095680401282

def StoE(spes):
    return CAL_a*spes**2 + CAL_b*spes + CAL_c

evtfnames   = ["/Users/jrenner/local/data/NEW/6342/15x15x15/npz_combined_6342.npz",
               "/Users/jrenner/local/data/NEW/6346/15x15x15/npz_combined_6346.npz",
               "/Users/jrenner/local/data/NEW/6347/15x15x15/npz_combined_6347.npz",
               "/Users/jrenner/local/data/NEW/6348/15x15x15/npz_combined_6348.npz",
               "/Users/jrenner/local/data/NEW/6349/15x15x15/npz_combined_6349.npz",
               "/Users/jrenner/local/data/NEW/6351/15x15x15/npz_combined_6351.npz",
               "/Users/jrenner/local/data/NEW/6352/15x15x15/npz_combined_6352.npz",
               "/Users/jrenner/local/data/NEW/6365/15x15x15/npz_combined_6365.npz",
               "/Users/jrenner/local/data/NEW/6482/15x15x15/npz_combined_6482.npz", 
               "/Users/jrenner/local/data/NEW/6483/15x15x15/npz_combined_6483.npz",
               "/Users/jrenner/local/data/NEW/6484/15x15x15/npz_combined_6484.npz",
               "/Users/jrenner/local/data/NEW/6485/15x15x15/npz_combined_6485.npz"]

kalpha_mu = [9073.87757,  # 6342
             9013.27925,  # 6346
             8991.11666,  # 6347
             8964.70222,  # 6348
             9002.96242,  # 6349
             8981.19232,  # 6351
             8997.61118,  # 6352
             9123.09052,  # 6365
             9461.06544,  # 6482
             9454.31340,  # 6483
             9457.98010,  # 6484
             9472.07755]  # 6485

## Load the information directly from files

`A_evtnum`      -- event number<br>
`A_time`        -- timestamp<br>
`A_eblob1`      -- energy of blob1 (most energetic) as obtained from Paolina analysis<br>
`A_eblob2`      -- energy of blob2 (least energetic) as obtained from Paolina analysis<br>
`A_emtrk`       -- energy of 10 most energetic Paolina tracks in the event<br>
`A_xmtrk`       -- average reconstructed $x$-coordinate of 10 most energetic Paolina tracks in the event<br>
`A_ymtrk`       -- average reconstructed $y$-coordinate of 10 most energetic Paolina tracks in the event<br>
`A_zmtrk`       -- average reconstructed $z$-coordinate of 10 most energetic Paolina tracks in the event<br>
`A_ntrks`       -- number of tracks in the event<br>
`A_lmtrk`       -- length of most energetic Paolina track in event<br>
`A_nvox`        -- number of voxels in the event<br>
`A_nhits`       -- number of hits in the event<br>
`A_Ec`          -- fully corrected energy (lifetime by hit + $(x,y)$ by hit)<br>
`A_Eczlen`      -- fully corrected energy + z-length correction<br>
`A_E0`          -- uncorrected energy<br>
`A_xavg`        -- average reconstructed $x$-coordinate of event<br>
`A_yavg`        -- average reconstructed $y$-coordinate of event<br>
`A_zavg`        -- average reconstructed $z$-coordinate of event<br>
`A_ravg`        -- average $r = \sqrt{x^2 + y^2}$ of event<br>
`A_xmin`        -- minimum reconstructed $x$-coordinate of event<br>
`A_ymin`        -- minimum reconstructed $y$-coordinate of event<br>
`A_zmin`        -- minimum reconstructed $z$-coordinate of event<br>
`A_xmax`        -- maximum reconstructed $x$-coordinate of event<br>
`A_ymax`        -- maximum reconstructed $y$-coordinate of event<br>
`A_zmax`        -- maximum reconstructed $z$-coordinate of event<br>
`A_rmin`        -- minimum value of $r = \sqrt{x^2 + y^2}$ over all hits in event<br>
`A_rmax`        -- maximum value of $r = \sqrt{x^2 + y^2}$ over all hits in event<br>

In [ ]:
A_evtnum = []; A_time = []; A_ES1 = []
A_eblob1 = []; A_eblob2 = []; A_emtrk = []; A_etrks = []; A_xtrks = []; A_ytrks = []; A_ztrks = []
A_ntrks = []; A_lmtrk = []; A_nvox = []; A_nhits = []
A_Ec = []; A_Eczlen = []; A_E0 = []
A_xavg = []; A_yavg = []; A_zavg = []; A_ravg = []
A_xmin = []; A_ymin = []; A_zmin = []
A_xmax = []; A_ymax = []; A_zmax = []
A_rmin = []; A_rmax = []
for ii,fname in enumerate(evtfnames):
    
    fscale = kalpha_mu[0]/kalpha_mu[ii]

    print("Adding file {}...".format(fname))

    fn = np.load(fname)
    A_evtnum = np.concatenate((A_evtnum,fn['A_evtnum']))
    A_time   = np.concatenate((A_time,fn['A_time']))
    A_ES1    = np.concatenate((A_ES1,fn['A_ES1']))
    A_eblob1 = np.concatenate((A_eblob1,fn['A_eblob1']))
    A_eblob2 = np.concatenate((A_eblob2,fn['A_eblob2']))
    A_ntrks  = np.concatenate((A_ntrks,fn['A_ntrks']))
    A_lmtrk  = np.concatenate((A_lmtrk,fn['A_lmtrk']))
    A_nvox   = np.concatenate((A_nvox,fn['A_nvox']))
    A_nhits  = np.concatenate((A_nhits,fn['A_nhits']))
    A_Ec     = np.concatenate((A_Ec,fscale*fn['A_Ec']))
    A_Eczlen = np.concatenate((A_Eczlen,fscale*fn['A_Eczlen']))
    A_E0     = np.concatenate((A_E0,fscale*fn['A_E0']))
    A_xavg   = np.concatenate((A_xavg,fn['A_xavg']))
    A_yavg   = np.concatenate((A_yavg,fn['A_yavg']))
    A_zavg   = np.concatenate((A_zavg,fn['A_zavg']))
    A_ravg   = np.concatenate((A_ravg,fn['A_ravg']))
    A_xmin   = np.concatenate((A_xmin,fn['A_xmin']))
    A_ymin   = np.concatenate((A_ymin,fn['A_ymin']))
    A_zmin   = np.concatenate((A_zmin,fn['A_zmin']))
    A_xmax   = np.concatenate((A_xmax,fn['A_xmax']))
    A_ymax   = np.concatenate((A_ymax,fn['A_ymax']))
    A_zmax   = np.concatenate((A_zmax,fn['A_zmax']))
    A_rmin   = np.concatenate((A_rmin,fn['A_rmin']))
    A_rmax   = np.concatenate((A_rmax,fn['A_rmax']))
    
    if(len(A_etrks) > 0): A_etrks  = np.concatenate((A_etrks,fscale*fn['A_emtrk']))
    else: A_etrks = fn['A_emtrk']
    
    if(len(A_xtrks) > 0): A_xtrks  = np.concatenate((A_xtrks,fn['A_xmtrk']))
    else: A_xtrks = fn['A_xmtrk']
    
    if(len(A_ytrks) > 0): A_ytrks  = np.concatenate((A_ytrks,fn['A_ymtrk']))
    else: A_ytrks = fn['A_ymtrk']
        
    if(len(A_ztrks) > 0): A_ztrks  = np.concatenate((A_ztrks,fn['A_zmtrk']))
    else: A_ztrks = fn['A_zmtrk']

        
# Calibrate the energy.
A_Ecal = CAL_a*A_Ec**2 + CAL_b*A_Ec + CAL_c
A_etrks = CAL_a*A_etrks**2 + CAL_b*A_etrks + CAL_c

# Produce an array with only the energy of the maximum track for each event.
A_emtrk = np.max(A_etrks,axis=1)
A_emtrk = CAL_a*A_emtrk**2 + CAL_b*A_emtrk + CAL_c

# Correct for z-length effect.
A_zlen = A_zmax - A_zmin
A_Eczlcorr = A_Ecal*(1 + 3.1e-4*A_zlen)

A_Ecplot = A_Eczlcorr

print("{} total events".format(len(A_evtnum)))


## Definition of key quantities

In [ ]:
# Key variables in constructing cuts.
Emin = StoE(0); Emax = StoE(800000)                  # basic cut energy range
eb1_low = 1260; eb1_high = 1320          # blob plot intervals
eb2_low = 1596; eb2_high = 1645
eb3_low = 1690; eb3_high = 2500
Emin_emtrk = StoE(1000); Emax_emtrk = StoE(2500)     # emtrk basic energy cut range

cwide_zmin = 50; cwide_zmax = 500
cwide_rmax = 180
ctight_zmin = 150; ctight_zmax = 300
ctight_rmax = 150

cs_zmin = cwide_zmin; cs_zmax = cwide_zmax
cs_rmax = cwide_rmax

# cs_zmin = 100; cs_zmax = 300
# cs_rmax = 150

xr_zmin = cwide_zmin; xr_zmax = cwide_zmax
xr_rmax = cwide_rmax

# xr_zmin = 100;   xr_zmax = 300
# xr_rmax = 150

th_zmin = cwide_zmin; th_zmax = cwide_zmax
th_rmax = cwide_rmax

# th_zmin = 100; th_zmax = 300
# th_rmax = 150

tl_zmin = cwide_zmin; tl_zmax = cwide_zmax
tl_rmax = cwide_rmax

# tl_zmin = 80; tl_zmax = 270
# tl_rmax = 155

# ------------------------------------------------------------------------------------------
# Pre-defined cuts
C_basic       = (A_E0 > 0) & (A_Ecplot > 0) & (A_zavg < 580)
C_wide        = C_basic & (A_zmin > cwide_zmin) & (A_zmax < cwide_zmax) & (A_rmax < cwide_rmax)
C_tight       = C_basic & (A_zmin > ctight_zmin) & (A_zmax < ctight_zmax) & (A_rmax < ctight_rmax)
C_tfiducial   = (abs(A_xmax) < 100) & (abs(A_ymax) < 100) & (abs(A_xmin) < 100) & (abs(A_ymin) < 100) & (A_zavg > 50) & (A_zavg < 150)

# Cuts defining different spectral regions
C_trks                 = (A_lmtrk > 90) & (A_lmtrk < 180) & (A_ntrks == 1)

C_basic_emtrk          = (A_emtrk > Emin_emtrk) & (A_emtrk < Emax_emtrk)
C_emtrk_regionprepeak  = C_basic_emtrk & (A_emtrk > eb1_low) & (A_emtrk < eb1_high)
C_emtrk_regionpeak     = C_basic_emtrk & (A_emtrk > eb2_low) & (A_emtrk < eb2_high)
C_emtrk_regionpostpeak = C_basic_emtrk & (A_emtrk > eb3_low) & (A_emtrk < eb3_high)

# Extract individual tracks
etrks_basic = A_etrks[C_basic].flatten()
xtrks_basic = A_xtrks[C_basic].flatten()
ytrks_basic = A_ytrks[C_basic].flatten()
ztrks_basic = A_ztrks[C_basic].flatten()
rtrks_basic = np.array([(xx**2 + yy**2)**0.5 for xx,yy in zip(xtrks_basic,ytrks_basic)])

# Peak energies
E_xKalpha = (29.112*0.00261 + 29.461*25.6 + 29.782*47.4)/(0.00261 + 25.6 + 47.4)
E_xKbeta  = (33.562*4.35 + 33.624*8.40 + 33.881*0.085 + 34.419*2.54 + 34.496*0.492)/(4.35 + 8.40 + 0.085 + 2.54 + 0.492)
E_Kr      = 41.543
E_Cs      = 661.657
E_descape = 1592.535
E_Tl      = 2614.533
print("""
E_xKalpha = {}
E_xKbeta = {}
E_Kr = {}
E_Cs = {}
E_descape = {}
E_Tl = {}
""".format(E_xKalpha,E_xKbeta,E_Kr,E_Cs,E_descape,E_Tl))

# S1-S2 cuts
mS1 = 3.6
bS1 = 50
bminusS1 = -50
bplusS1 = 150
#C_S1S2 = ((A_Ec*CAL_m + CAL_b) < (mS1*A_ES1 + bplusS1)) & ((A_Ec*CAL_m + CAL_b) > (mS1*A_ES1 + bminusS1))

print("Events in key quantities: {0}".format(len(A_eblob1)))

--------------------------

# Energy resolution analysis

## Uncorrected energy

In [ ]:
fig = plt.figure(1)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

ax1 = fig.add_subplot(121)
y, x, _ = plt.hist(A_E0[C_basic & (A_Ecplot > 650) & (A_Ecplot < 675)], 200, range=[1.4e5,2.0e5], **hargs)
plt.xlabel('Photons')
plt.ylabel('Counts/bin')
plt.title("Cs peak")

ax2 = fig.add_subplot(122)
y, x, _ = plt.hist(A_E0[C_basic & (A_Ecplot > 1580) & (A_Ecplot < 1650)], 200, range=[2.5e5,5e5], **hargs)
plt.xlabel('Photons')
plt.ylabel('Counts/bin')
plt.title("Double-escape peak")

## Energy spectrum (full)

In [ ]:
cuts_fullspectrum = C_wide

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

y, x, _ = plt.hist(A_Ecplot[cuts_fullspectrum], 200, range=[StoE(30000),StoE(9.5e5)], **hargs)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')
plt.yscale('log')
plt.annotate('$^{137}$Cs photopeak', xy=(StoE(220000), 30000),  xycoords='data',
                 xytext=(StoE(280000), 60000), textcoords='data',
                 arrowprops=dict(arrowstyle="-"))
plt.annotate('$^{208}$Tl e$^{+}$e$^{-}$ double-escape', xy=(StoE(530000), 8000),  xycoords='data',
                 xytext=(StoE(580000), 28000), textcoords='data',
                 arrowprops=dict(arrowstyle="-"))
plt.annotate('$^{208}$Tl photopeak', xy=(StoE(840000), 600),  xycoords='data',
                xytext=(StoE(680000), 4000), textcoords='data',
                arrowprops=dict(arrowstyle="-"))

plt.savefig("{}/CSTH_espectrum_full.{}".format(save_dir,ftype), bbox_inches='tight')

## Energy spectrum (two regions)

In [ ]:
# Energy spectrum, focusing on 2.6 MeV peak
nbins = 200

cuts_regionCs = C_wide
cuts_regionTl = C_wide

plt_rng_Cs = [StoE(180000),StoE(215000)]
nbins_Cs = 150

plt_rng_Tl = [StoE(450000),StoE(550000)]
nbins_Tl = 150

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(15.0)

# Cs region
ax1 = fig.add_subplot(121)
y, x, _ = plt.hist(A_Ecplot[cuts_regionCs], nbins, range=plt_rng_Cs, **hargs)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')

# Tl region
ax2 = fig.add_subplot(122)
y, x, _ = plt.hist(A_Ecplot[cuts_regionTl], nbins, range=plt_rng_Tl, **hargs)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')

plt.savefig("{}/CSTL_espectrum_2regions.{}".format(save_dir,ftype), bbox_inches='tight')

## S1 vs. S2

In [ ]:
nxybins = 30

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(15.0)

print(mpl.cm.get_cmap('jet'))

s1_low = 300
s1_high = 550
s2_low = 1550
s2_high = 1650

# S1 vs. corrected S2
ax1 = fig.add_subplot(121)
h, x, y = np.histogram2d(-A_Ecplot[C_wide], A_ES1[C_wide], bins=[nxybins, nxybins], range=[[-s2_high,-s2_low],[s1_low,s1_high]])
plt.imshow(h, extent=[s1_low,s1_high,s2_low,s2_high], interpolation = "none", cmap='jet', aspect=(s1_high-s1_low)/(s2_high-s2_low), norm=SymLogNorm(linthresh=1.0, linscale=1.0))
#plt.plot(np.arange(0,500),mS1*np.arange(0,500)+bS1,color='red',linewidth=2)
#plt.plot(np.arange(0,450),mS1*np.arange(0,450)+bplusS1,color='red',linewidth=2,linestyle='--')
#plt.plot(np.arange(50,550),mS1*np.arange(50,550)+bminusS1,color='red',linewidth=2,linestyle='--')
plt.plot()
plt.colorbar()
plt.xlabel('S1 (pes)')
plt.ylabel('E (keV)')

# S1 vs. uncorrected S2
ax2 = fig.add_subplot(122)
h, x, y = np.histogram2d(-A_E0[C_wide], A_ES1[C_wide], bins=[nxybins, nxybins], range=[[-800000,0],[0,1000]])
plt.imshow(h, extent=[0,1000,0,800000], interpolation = "gaussian", cmap='jet', aspect=1000./800000, norm=SymLogNorm(linthresh=1.0, linscale=1.0))
plt.colorbar()
plt.xlabel('S1 (pes)')
plt.ylabel('E$_{uncorrected}$ (pes)')

plt.savefig("{}/CSTL_S1_vs_S2.{}".format(save_dir,ftype), bbox_inches='tight')

## S2 corrected vs. uncorrected

In [ ]:
nxybins = 200

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(15.0)

print(mpl.cm.get_cmap('jet'))

# S1 vs. corrected S2
h, x, y = np.histogram2d(-A_Ecplot[C_wide], A_E0[C_wide], bins=[nxybins, nxybins], range=[[-StoE(800000),0],[0,800000]])
plt.imshow(h, extent=[0,800000,0,StoE(800000)], interpolation = "gaussian", cmap='jet', aspect=800000./StoE(800000), norm=SymLogNorm(linthresh=1.0, linscale=1.0))
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.colorbar()
plt.xlabel('S2$_{uncorrected}$ (pes)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_S2corr_vs_S2uncorr.{}".format(save_dir,ftype), bbox_inches='tight')

## Energy vs. time

In [ ]:
# Energy vs. time
cuts_T = C_wide

nbins = 8

print(A_time[0])
print(A_time[-1])

# for double-escape peak
Thist_min_D = A_time[0]
Thist_max_D = A_time[-1]
Ehist_min_D = 1575
Ehist_max_D = 1615
plt_rng_Tlow_D = A_time[0]
plt_rng_Thigh_D = A_time[-1]
plt_rng_Elow_D = 1400
plt_rng_Ehigh_D = 1800

# for Cs peak
Thist_min_Cs = A_time[0]
Thist_max_Cs = A_time[-1]
Ehist_min_Cs = 648
Ehist_max_Cs = 675
plt_rng_Tlow_Cs = A_time[0]
plt_rng_Thigh_Cs = A_time[-1]
plt_rng_Elow_Cs = 600
plt_rng_Ehigh_Cs = 700

Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_time[cuts_T],A_Ecplot[cuts_T],nbins=nbins,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_time[cuts_T],A_Ecplot[cuts_T],nbins=nbins,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))

prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(13.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(221);
ax1.scatter(A_time[cuts_T],A_Ecplot[cuts_T],s=0.5)
ax1.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.title("Energy vs. T")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('T')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(222);
ax2.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax2.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. T profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('T')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(223);
ax5.scatter(A_time[cuts_T],A_Ecplot[cuts_T],s=0.5)
ax5.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.title("Energy vs. T")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('T')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(224);
ax6.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax6.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. T profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('T')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_T.{}".format(save_dir,ftype), bbox_inches='tight')

## Energy vs. z-length

In [ ]:
# Energy vs. z-length
cuts_T = C_wide & (A_ntrks == 1)

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 34
Ehist_min_Cs = 645
Ehist_max_Cs = 672
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 640
plt_rng_Ehigh_Cs = 680

# for double-escape peak
Thist_min_D = 24
Thist_max_D = 72
Ehist_min_D = 1575
Ehist_max_D = 1615
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1630

# for Tl peak
Thist_min_Tl = 40
Thist_max_Tl = 150
Ehist_min_Tl = 2578
Ehist_max_Tl = 2645
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = 2520
plt_rng_Ehigh_Tl = 2660

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(flinear, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],1.)
pfit_Cs = np.poly1d([m_Cs, b_Cs])
print("Cs peak, slope for correction: {}; intercept: {}".format(m_Cs, b_Cs))

p_D, V_D = curve_fit(flinear, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],1.)
pfit_D = np.poly1d([m_D, b_D])
print("Double-escape peak: slope for correction {}; intercept: {}".format(m_D, b_D))

p_Tl, V_Tl = curve_fit(flinear, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],1.)
pfit_Tl = np.poly1d([m_Tl, b_Tl])
print("Tl peak: slope for correction {}; intercept: {}".format(m_Tl, b_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-A_Ecplot[cuts_T], A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m/b = {:.2f} $\\times 10^{{-4}}$ mm$^{{-1}}$".format(m_Cs/b_Cs*1e4))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-A_Ecplot[cuts_T], A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m/b = {:.2f} $\\times 10^{{-4}}$ mm$^{{-1}}$".format(m_D/b_D*1e4))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-A_Ecplot[cuts_T], A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m/b = {:.2f} $\\times 10^{{-4}}$ mm$^{{-1}}$".format(m_Tl/b_Tl*1e4))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_zlen_keV.{}".format(save_dir,"png"), bbox_inches='tight')

## x-y, and distributions

In [ ]:
cuts_xyz_Cs = C_basic & (A_Ecplot > 645) & (A_Ecplot < 675)

cuts_xyz = cuts_xyz_Cs
nxybins = 25
nzbins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

# x-y distribution
ax1 = fig.add_subplot(121)
h, x, y = np.histogram2d(-A_yavg[cuts_xyz], A_xavg[cuts_xyz], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
ctight_circle = plt.Circle((0, 0), cs_rmax, color='gray', alpha=0.15)
ctight_lcircle = plt.Circle((0, 0), cs_rmax, facecolor='none', linestyle='solid', edgecolor='red',linewidth=2)
plt.gca().add_artist(ctight_lcircle)

# z distribution
ax2 = fig.add_subplot(122)
hz = plt.hist(A_zavg[cuts_xyz], bins=nzbins, range=[0,580], **hargs)
plt.xlabel("average z (mm)")
plt.axvline(cs_zmin,linestyle='solid',color='red',linewidth=1.2)
plt.axvline(cs_zmax,linestyle='solid',color='red',linewidth=1.2)
plt.axvspan(cs_zmin,cs_zmax,linestyle='solid',facecolor='gray',edgecolor='red',linewidth=1.2,alpha=0.05)

plt.savefig("{}/CSTH_xyzdist_Cs.{}".format(save_dir,ftype), bbox_inches='tight')

In [ ]:
cuts_xyz_Th = C_basic & (A_Ecplot > 1575) & (A_Ecplot < 1615)

cuts_xyz = cuts_xyz_Th
nxybins = 25
nzbins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

# x-y distribution
ax1 = fig.add_subplot(121)
h, x, y = np.histogram2d(-A_yavg[cuts_xyz], A_xavg[cuts_xyz], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
ctight_circle = plt.Circle((0, 0), th_rmax, color='gray', alpha=0.15)
ctight_lcircle = plt.Circle((0, 0), th_rmax, facecolor='none', linestyle='solid', edgecolor='red',linewidth=2)
plt.gca().add_artist(ctight_lcircle)

# z distribution
ax2 = fig.add_subplot(122)
hz = plt.hist(A_zavg[cuts_xyz], bins=nzbins, range=[0,580], **hargs)
plt.xlabel("average z (mm)")
plt.axvline(th_zmin,linestyle='solid',color='red',linewidth=1.2)
plt.axvline(th_zmax,linestyle='solid',color='red',linewidth=1.2)
plt.axvspan(th_zmin,th_zmax,linestyle='solid',facecolor='gray',edgecolor='red',linewidth=1.2,alpha=0.15)

plt.savefig("{}/CSTH_xyzdist_Th.{}".format(save_dir,ftype), bbox_inches='tight')

In [ ]:
cuts_xyz_xray = (etrks_basic > 27.5) & (etrks_basic < 32)

cuts_xyz = cuts_xyz_xray
nxybins = 25
nzbins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

# x-y distribution
ax1 = fig.add_subplot(121)
h, x, y = np.histogram2d(-ytrks_basic[cuts_xyz], xtrks_basic[cuts_xyz], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
ctight_circle = plt.Circle((0, 0), xr_rmax, color='gray', alpha=0.15)
ctight_lcircle = plt.Circle((0, 0), xr_rmax, facecolor='none', linestyle='solid', edgecolor='red',linewidth=2)
plt.gca().add_artist(ctight_lcircle)

# z distribution
ax2 = fig.add_subplot(122)
hz = plt.hist(ztrks_basic[cuts_xyz], bins=nzbins, range=[0,580], **hargs)
plt.xlabel("average z (mm)")
plt.axvline(xr_zmin,linestyle='solid',color='red',linewidth=1.2)
plt.axvline(xr_zmax,linestyle='solid',color='red',linewidth=1.2)
plt.axvspan(xr_zmin,xr_zmax,linestyle='solid',facecolor='gray',edgecolor='red',linewidth=1.2,alpha=0.15)

plt.savefig("{}/CSTH_xyzdist_xray.{}".format(save_dir,ftype), bbox_inches='tight')

In [ ]:
cuts_xyz_tl =  C_basic & (A_Ecplot > 2575) & (A_Ecplot < 2650)

cuts_xyz = cuts_xyz_tl
nxybins = 25
nzbins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

# x-y distribution
ax1 = fig.add_subplot(121)
h, x, y = np.histogram2d(-A_yavg[cuts_xyz], A_xavg[cuts_xyz], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
ctight_circle = plt.Circle((0, 0), tl_rmax, color='gray', alpha=0.15)
ctight_lcircle = plt.Circle((0, 0), tl_rmax, facecolor='none', linestyle='solid', edgecolor='red',linewidth=2)
plt.gca().add_artist(ctight_lcircle)

# z distribution
ax2 = fig.add_subplot(122)
hz = plt.hist(A_zavg[cuts_xyz], bins=nzbins, range=[0,580], **hargs)
plt.xlabel("average z (mm)")
plt.axvline(tl_zmin,linestyle='solid',color='red',linewidth=1.2)
plt.axvline(tl_zmax,linestyle='solid',color='red',linewidth=1.2)
plt.axvspan(tl_zmin,tl_zmax,linestyle='solid',facecolor='gray',edgecolor='red',linewidth=1.2,alpha=0.15)

plt.savefig("{}/CSTH_xyzdist_Tl.{}".format(save_dir,ftype), bbox_inches='tight')

## Energy spectrum (x-rays)

In [ ]:
# fit xray peaks with and without profile corrections
rmax = xr_rmax
zmin = xr_zmin; zmax = xr_zmax
xraycuts = (rtrks_basic < rmax) & (ztrks_basic > zmin) & (ztrks_basic < zmax)

plt_rng = [StoE(7000), StoE(12000)]
nbins = 90
nxybins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

fit_rng_low = StoE(7800)
fit_rng_high = StoE(12000)
text_x = StoE(7000)

fit_gauss_A1 = 3000
fit_gauss_mu1 = StoE(9000)
fit_gauss_sigma1 = StoE(300)
fit_gauss_A2 = 1200
fit_gauss_mu2 = StoE(10200)
fit_gauss_sigma2 = StoE(300)

ax1 = fig.add_subplot(121)
y0, x0, _ = plt.hist(etrks_basic[xraycuts], nbins, range=plt_rng, **hargs)

# 2-Gaussian + polynomial fit
x    = x0[:-1] + np.diff(x0) * 0.5
sel  = coref.in_range(x, fit_rng_low, fit_rng_high)
x, y = x[sel], y0[sel]
f    = fitf.fit(lambda x, *args: gauss2(x, *args[:6]) + fitf.polynom(x, *args[6:]), x, y, (fit_gauss_A1, fit_gauss_mu1, fit_gauss_sigma1, fit_gauss_A2, fit_gauss_mu2, fit_gauss_sigma2, 1., -1./fit_gauss_mu1, -1./fit_gauss_mu1**2))
#f    = fitf.fit(gauss2, x, y, (fit_gauss_A1, fit_gauss_mu1, fit_gauss_sigma1, fit_gauss_A2, fit_gauss_mu2, fit_gauss_sigma2))
plt.plot(x, f.fn(x), "r")
plt.text(text_x, 2.0*max(y)/6, gauss2text(f.values[0:6],E_xKalpha,E_xKbeta), fontsize=12)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')
#plt.title("Rmax < {}, {} < Zmax < {}".format(rmax,zmin,zmax), fontsize=12, fontweight='bold')
print("Fit parameters for wide cuts (amp, mu, sigma)")
print(f.values)
print("Fit errors for wide cuts (amp, mu, sigma)")
print(f.errors)

xyz_rng_low = 28
xyz_rng_high = 32

ax4 = fig.add_subplot(122)
h, x, y = np.histogram2d(-ytrks_basic[xraycuts & (etrks_basic > xyz_rng_low) & (etrks_basic < xyz_rng_high)], xtrks_basic[xraycuts & (etrks_basic > xyz_rng_low) & (etrks_basic < xyz_rng_high)], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
plt.title("x,y for $E\in ({},{})$ keV".format(xyz_rng_low,xyz_rng_high), fontsize=14)

# Save relevant fit parameters for later use
mu_xray_fid      = f.values[1] 
e_mu_xray_fid    = f.errors[1]
sigma_xray_fid   = f.values[2]
e_sigma_xray_fid = f.errors[2]

mu_xray2_fid      = f.values[4]
e_mu_xray2_fid    = f.errors[4]
sigma_xray2_fid   = f.values[5]
e_sigma_xray2_fid = f.errors[5]

plt.savefig("{}/CSTH_espectrum_xray_fit.{}".format(save_dir,ftype), bbox_inches='tight')

# Energy spectrum (Cs 662 keV peak)

In [ ]:
rmax = cs_rmax
zmin = cs_zmin; zmax = cs_zmax
rcuts = C_basic & (A_rmax < rmax) & (A_zmin > zmin) & (A_zmax < zmax)

text_x = StoE(192000)
plt_rng = [StoE(190000),StoE(220000)]
nbins = 120
nxybins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

fit_rng_low = StoE(202000)
fit_rng_high = StoE(230900)
fit_gauss_A = 6000 #95
fit_gauss_mu = StoE(208800)
fit_gauss_sigma = StoE(3200)
fit_exp_A = 50
fit_exp_mean = StoE(3.0e4)
fit_exp_x0 = StoE(1.6e5)

ax1 = fig.add_subplot(121)
y0, x0, _ = plt.hist(A_Ecplot[rcuts], nbins, range=plt_rng, **hargs)

# Gaussian fit
x    = x0[:-1] + np.diff(x0) * 0.5
sel  = coref.in_range(x, fit_rng_low, fit_rng_high)
x, y = x[sel], y0[sel]
f    = fitf.fit(lambda x, *args: fitf.gauss(x, *args[:3]) + fitf.polynom(x, *args[3:]), x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, 1, -1/fit_gauss_mu, -1/fit_gauss_mu**2))
#f    = fitf.fit(gaussexpo, x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, fit_exp_A, fit_exp_mean, fit_exp_x0))
plt.plot(x, f.fn(x), "r")
plt.text(text_x, 3.5*max(y)/6, gausstext(f.values[0:3],E_Cs),fontsize=14)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')
#plt.title("Rmax < {}, {} < Zmax < {}".format(rmax,zmin,zmax), fontsize=12, fontweight='bold')
#print("Fit parameters for wide cuts (amp, mu, sigma, A, mean, x0)")
#print(f.values)
#print("Fit errors for wide cuts (amp, mu, sigma)")
#print(f.errors)

xyz_rng_low = 645
xyz_rng_high = 675

ax4 = fig.add_subplot(122) 
h, x, y = np.histogram2d(-A_yavg[rcuts & (A_Ecplot > xyz_rng_low) & (A_Ecplot < xyz_rng_high)], A_xavg[rcuts & (A_Ecplot > xyz_rng_low) & (A_Ecplot < xyz_rng_high)], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
plt.title("x,y for $E\in ({},{})$".format(xyz_rng_low,xyz_rng_high), fontsize=14)

# Save relevant fit parameters for later use
mu_cs_fid      = f.values[1] 
e_mu_cs_fid    = f.errors[1]
sigma_cs_fid   = f.values[2]
e_sigma_cs_fid = f.errors[2]

plt.savefig("{}/CSTH_espectrum_Cs_fit.{}".format(save_dir,ftype), bbox_inches='tight')

### Save list of 137Cs peak event numbers within some energy range

In [ ]:
mu_E = mu_cs_fid
sigma_E = sigma_cs_fid
print("mu = {}, sigma = {}".format(mu_E,sigma_E))

cut_Ei = mu_E - sigma_E*3
cut_Ef = mu_E + sigma_E*3
cuts_evtnum = C_wide & ((A_Ecplot*CAL_m + CAL_b) > cut_Ei) & ((A_Ecplot*CAL_m + CAL_b) < cut_Ef)

save_evtnum = A_evtnum[cuts_evtnum].astype('int')
print("Saving {} event numbers for events with energy in range {} to {}...".format(len(save_evtnum),cut_Ei,cut_Ef))
np.savez("cs_evts_6206.npz", A_evtnum=save_evtnum)
print("Done.")

# Energy spectrum (208Tl double-escape peak)

In [ ]:
# fit double-escape peak with and without profile corrections
rmax = th_rmax
zmin = th_zmin; zmax = th_zmax
rcuts = C_basic & (A_rmax < rmax) & (A_zmin > zmin) & (A_zmax < zmax)

text_x = StoE(492000)
plt_rng = [StoE(490000),StoE(530000)]
nbins = 65
nxybins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

fit_rng_low = StoE(492000)
fit_rng_high = StoE(528000)
fit_gauss_A = 2000
fit_gauss_mu = StoE(515000)
fit_gauss_sigma = StoE(2800)
fit_exp_A = 80
fit_exp_mean = StoE(1.16e5)
fit_exp_x0 = StoE(2.1e2)

ax1 = fig.add_subplot(121)
y0, x0, _ = plt.hist(A_Ecplot[rcuts], nbins, range=plt_rng, **hargs)
#y0, x0, _ = plt.hist(1592*(A_Ec[rcuts]*CAL_m + CAL_b)/prof_D(A_zmax[rcuts] - A_zmin[rcuts]), nbins, range=plt_rng, **hargs)

# Gaussian fit
x    = x0[:-1] + np.diff(x0) * 0.5
sel  = coref.in_range(x, fit_rng_low, fit_rng_high)
x, y = x[sel], y0[sel]
f    = fitf.fit(gaussexpo, x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, fit_exp_A, fit_exp_mean, fit_exp_x0))
plt.plot(x, f.fn(x), "r")
plt.text(text_x, 3.5*max(y)/6, gaussexpotext(f.values,E_descape), fontsize=14)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')
#plt.title("Rmax < {}, {} < Zmax < {}".format(rmax,zmin,zmax), fontsize=12, fontweight='bold')
print("Fit parameters (amp , mu, sigma, A, mean, x0)")
print(f.values)
print("Fit errors (amp, mu, sigma, A, mean, x0)")
print(f.errors)
print(len(A_Ecplot[rcuts & (A_Ecplot > 1550) & (A_Ecplot < 1650)]))

xyz_rng_low = 1575
xyz_rng_high = 1640

ax4 = fig.add_subplot(122) 
h, x, y = np.histogram2d(-A_yavg[rcuts & (A_Ecplot > xyz_rng_low) & (A_Ecplot < xyz_rng_high)], A_xavg[rcuts & (A_Ecplot > xyz_rng_low) & (A_Ecplot < xyz_rng_high)], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
plt.title("x,y for $E\in ({},{})$".format(xyz_rng_low,xyz_rng_high), fontsize=14)

# Save relevant fit parameters for later use
mu_de_fid      = f.values[1] 
e_mu_de_fid    = f.errors[1]
sigma_de_fid   = f.values[2]
e_sigma_de_fid = f.errors[2]

plt.savefig("{}/CSTH_espectrum_doubleescape_fit.{}".format(save_dir,ftype), bbox_inches='tight')

### Save list of double-escape peak event numbers within some energy range

In [ ]:
mu_E = mu_de_fid
sigma_E = sigma_de_fid
print("mu = {}, sigma = {}".format(mu_E,sigma_E))

cut_Ei = mu_E - sigma_E*3
cut_Ef = mu_E + sigma_E*3
cuts_evtnum = C_wide & ((A_Ecplot*CAL_m + CAL_b) > cut_Ei) & ((A_Ecplot*CAL_m + CAL_b) < cut_Ef)

save_evtnum = A_evtnum[cuts_evtnum].astype('int')
print("Saving {} event numbers for events with energy in range {} to {}...".format(len(save_evtnum),cut_Ei,cut_Ef))
np.savez("descape_evts_6351.npz", A_evtnum=save_evtnum)
print("Done.")

# Energy spectrum (208Tl photopeak)

In [ ]:
# fit 208Tl photopeak for paper
rmax = tl_rmax
zmin = tl_zmin; zmax = tl_zmax
rcuts = C_wide & (A_rmax < 155) & (A_zmin > 80) & (A_zmax < 270)

text_x = StoE(812000)
plt_rng = [StoE(810000),StoE(880000)]
nbins = 40
nxybins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

fit_rng_low = StoE(825000)
fit_rng_high = StoE(860000)
fit_gauss_A = 30
fit_gauss_mu = StoE(855000)
fit_gauss_sigma = StoE(20000)

ax1 = fig.add_subplot(121)
y0, x0, _ = plt.hist(A_Ecplot[rcuts], nbins, range=plt_rng, **hargs)
#y0, x0, _ = plt.hist(2614*(A_Ec[rcuts]*CAL_m + CAL_b)/prof_Tl(A_zmax[rcuts] - A_zmin[rcuts]), nbins, range=plt_rng, **hargs)

# Gaussian fit
x    = x0[:-1] + np.diff(x0) * 0.5
sel  = coref.in_range(x, fit_rng_low, fit_rng_high)
x, y = x[sel], y0[sel]
f    = fitf.fit(lambda x, *args: fitf.gauss(x, *args[:3]) + fitf.polynom(x, *args[3:]), x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, 1, -1/fit_gauss_mu, -1/fit_gauss_mu**2))
#f    = fitf.fit(gaussexpo, x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, fit_exp_A, fit_exp_mean, fit_exp_x0))
plt.plot(x, f.fn(x), "r")
plt.text(text_x, 3.8*max(y)/6, gausstext(f.values[0:3],E_Tl),fontsize=14)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')
#plt.title("Rmax < {}, {} < Zmax < {}".format(rmax,zmin,zmax), fontsize=12, fontweight='bold')
print("Fit parameters for wide cuts (amp, mu, sigma, A, mean, x0)")
print(f.values)
print("Fit errors for wide cuts (amp, mu, sigma)")
print(f.errors)

xyz_rng_low = 2600
xyz_rng_high = 2720

ax4 = fig.add_subplot(122) 
h, x, y = np.histogram2d(-A_yavg[rcuts & (A_Ecplot > xyz_rng_low) & (A_Ecplot < xyz_rng_high)], A_xavg[rcuts & (A_Ecplot > xyz_rng_low) & (A_Ecplot < xyz_rng_high)], bins=[nxybins, nxybins], range=[[-215, 215], [-215, 215]])
plt.imshow(h, extent=[x[0],x[-1],y[0],y[-1]], interpolation = "gaussian", cmap='jet')
plt.colorbar()
plt.xlabel("average x (mm)")
plt.ylabel("average y (mm)")
plt.title("x,y for $E\in ({},{})$".format(xyz_rng_low,xyz_rng_high), fontsize=14)

# Save relevant fit parameters for later use
mu_tl_fid      = f.values[1] 
e_mu_tl_fid    = f.errors[1]
sigma_tl_fid   = f.values[2]
e_sigma_tl_fid = f.errors[2]

plt.savefig("{}/CSTH_espectrum_Tl_photopeak_fit.{}".format(save_dir,ftype), bbox_inches='tight')

### Save list of Th photopeak event numbers within some energy range

In [ ]:
A_xlen = (A_xmax - A_xmin)
A_ylen = (A_ymax - A_ymin)
A_zlen = (A_zmax - A_zmin)
A_trklen = (A_xlen**2 + A_ylen**2 + A_zlen**2)**0.5
radtodeg = (180/np.pi)

mu_E = mu_tl_fid
sigma_E = sigma_tl_fid
print("mu = {}, sigma = {}".format(mu_E,sigma_E))

cut_Ei = mu_E - sigma_E*8
cut_Ef = mu_E + sigma_E*8
cuts_evtnum = C_wide & ((A_Ecplot*CAL_m + CAL_b) > cut_Ei) & ((A_Ecplot*CAL_m + CAL_b) < cut_Ef) #& (A_ntrks == 1) & (A_trklen > 0) & (radtodeg*np.arccos(A_zlen/A_trklen) > 0.7*radtodeg) & (radtodeg*np.arccos(A_zlen/A_trklen) < 1.4*radtodeg)

save_evtnum = A_evtnum[cuts_evtnum].astype('int')
save_dict = {}
save_dict[6485] = save_evtnum
print("Saving {} event numbers for events with energy in range {} to {}...".format(len(save_evtnum),cut_Ei,cut_Ef))
np.savez("photopeakTh_evts_6485.npz", evtdict=save_dict)
print("Done.")

---

## Energy calibration

** Xenon x-rays** (from [Table of Radioactive Isotopes](http://nucleardata.nuclear.lu.se/toi/xray.asp?act=list&el=Xe))
```
	                    Intensity per 100 vacancies in the
Assignment  E (keV)     K-shell	
Xe Ka3      29,112      0.00261 (8)
Xe Ka2      29,461      25.6 (6)
Xe Ka1      29,782      47.4 (11)
Xe Kb3      33,562      4.35 (10)
Xe Kb1      33,624      8.40 (19)
Xe Kb5      33,881      0.085 (4)
Xe Kb2      34,419      2.54 (6)
Xe Kb4      34,496      0.492 (20)
```

**Relevant fit points**
- Xenon K-alpha x-rays: intensity-weighted average of the 3 x-rays K$_{\alpha,1}$ - K$_{\alpha,3}$ listed above (29-30 keV)
- Xenon K-beta x-rays: intensity-weighted average of the 5 x-rays K$_{\beta,1}$ - K$_{\beta,5}$ listed above (33.5-34.5 keV)
- $^{137}$Cs gamma photopeak: 661.657 keV
- $^{208}$Tl gamma double-escape peak: 1592.535 keV

**Calibration function**
The calibration function is a line that gives the calibrated energy as a function of the uncalibratred energy: 

> E$_{cal}$ = m*$E_{uncal}$ + b

In [ ]:
mu_cal = np.array([E_xKalpha, E_Cs, E_descape, E_Tl])
sigma_cal = np.array([0.02, 0.07, 0.12, 0.3])
mu_uncal = np.array([mu_xray_fid, mu_cs_fid, mu_de_fid, mu_tl_fid])
sigma_uncal = np.array([e_mu_xray_fid, e_mu_cs_fid, e_mu_de_fid, e_mu_tl_fid])
print("mu_uncal = ", mu_uncal)
print("sigma_uncal = ", sigma_uncal)
print("sigma_uncal/mu_uncal*100 = ", sigma_uncal/mu_uncal*100)


In [ ]:
p, V = curve_fit(poly2, mu_uncal, mu_cal, sigma=sigma_cal, absolute_sigma=True)
a = p[0]
b = p[1]
c = p[2]
print("Fit parameters are: a = {}, b = {}, c = {}".format(a, b, c))

xfit = np.arange(0,1500000,10.) #np.arange(0,mu_uncal[-1]*1.01,10.)
pfit = np.poly1d([a, b, c])
sigma_E  = np.array([0, 0, 0, 0])

xres = xfit
res = 100*(mu_cal-pfit(mu_uncal))/mu_cal
res_err = 100*sigma_E/mu_cal
print("Residuals (%): {}".format(res))
#print("Residual error: {}".format(res_err))

fig = plt.figure()
fig.set_figheight(6.0)
fig.set_figwidth(15.0)
gs = gridspec.GridSpec(2, 1, height_ratios=[1,1]) 

ax1 = fig.add_subplot(gs[0])
ax1.plot(xfit, pfit(xfit), '--', color='gray')
ax1.plot(mu_uncal, mu_cal, '.', markerfacecolor='none', markersize=12, color='red')
#ax1.errorbar(mu_uncal, mu_cal, xerr=sigma_uncal, markersize=8, color='red', fmt='.')
#ax1.fill_between(xfit, pfit_minus(xfit), pfit_plus(xfit), facecolor='lightgray')
ax1.set_ylabel('Nominal energy, $E_0$ (keV)')
ax1.set_xlim(-20000,1500000)
#ax1.set_ylim(-0.02,1.02)

ax2 = fig.add_subplot(gs[1])
#ax2.fill_between(xres, 100*pfit_resplus(xres)/pfit(xres), 100*pfit_resminus(xres)/pfit(xres), facecolor='lightgray')
#ax2.errorbar(mu_uncal, res, yerr=res_err, capsize=5, fmt='.', markersize=6, color='red')
ax2.plot(mu_uncal, res, '.', markersize=6, color='red')
ax2.hlines(0.0,xmin=0.0,xmax=1500000,linestyle='--',color='gray',alpha=0.4)
ax2.set_xlabel('Raw energy, $Q$ (photoelectrons)')
ax2.set_ylabel('Residual (%)')
ax2.set_xlim(-20000,1500000)
ax2.set_ylim(-0.3,0.3)

plt.savefig("{}/CSTH_energy_calibration.{}".format(save_dir,ftype), bbox_inches='tight')

In [ ]:
# For two points only:
#m = (mu_cal[1] - mu_cal[0])/(mu_uncal[1] - mu_uncal[0])
#b = mu_cal[0] - m*mu_uncal[0]

p, V = curve_fit(flinear, mu_cal, mu_uncal, sigma=sigma_uncal, absolute_sigma=True)
m = p[0]
b = p[1]
sigma_m = V[0,0]**0.5
sigma_b = V[1,1]**0.5
sigma2_mb = V[0,1]

mstar = 1./m
bstar = -b/m
sigma_mstar = sigma_m/m**2
sigma_bstar = bstar*((sigma_b/b)**2 + (sigma_m/m)**2 + 2*sigma2_mb/(m*b))**0.5

fscale_m = 1.
fscale_b = 1.
print("Fit: m = {} +/- {}, b = {} +/- {}".format(mstar,sigma_mstar,bstar,sigma_bstar))
print("Calibration: m = {} +/- {}, b = {} +/- {}".format(mstar*fscale_m,sigma_mstar*fscale_m,bstar*fscale_b,sigma_bstar*fscale_b))

xfit = np.arange(0,mu_uncal[-1]*1.01,10.)#np.arange(0,1.01,0.01)
pfit = np.poly1d([mstar,bstar])
sigma_E  = pfit(mu_uncal)*((sigma_m/m)**2 + (sigma_b/(mu_uncal-p[1]))**2 + (2*sigma2_mb/(m*(mu_uncal-b))) + (sigma_uncal/(mu_uncal-b))**2)**0.5

xres = xfit
res = 100*(mu_cal-pfit(mu_uncal))/mu_cal
res_err = 100*sigma_E/mu_cal
print("Residual: {}".format(res))
print("Residual error: {}".format(res_err))

fig = plt.figure()
fig.set_figheight(6.0)
fig.set_figwidth(15.0)
gs = gridspec.GridSpec(2, 1, height_ratios=[1,1]) 

ax1 = fig.add_subplot(gs[0])
ax1.plot(xfit, pfit(xfit), '--', color='gray')
ax1.plot(mu_uncal, mu_cal, '.', markerfacecolor='none', markersize=12, color='red')
#ax1.errorbar(mu_uncal, mu_cal, xerr=sigma_uncal, markersize=8, color='red', fmt='.')
#ax1.fill_between(xfit, pfit_minus(xfit), pfit_plus(xfit), facecolor='lightgray')
ax1.set_ylabel('Nominal energy, $E_0$ (keV)')
ax1.set_xlim(-20000,900000)
#ax1.set_ylim(-0.02,1.02)

ax2 = fig.add_subplot(gs[1])
#ax2.fill_between(xres, 100*pfit_resplus(xres)/pfit(xres), 100*pfit_resminus(xres)/pfit(xres), facecolor='lightgray')
ax2.errorbar(mu_uncal, res, yerr=res_err, capsize=5, fmt='.', markersize=6, color='red')
ax2.hlines(0.0,xmin=0.0,xmax=850000,linestyle='--',color='gray',alpha=0.4)
ax2.set_xlabel('Raw energy, $Q$ (photoelectrons)')
ax2.set_ylabel('Residual (%)')
ax2.set_xlim(-20000,900000)
ax2.set_ylim(-3.0,3.0)

plt.savefig("{}/CSTH_energy_calibration.{}".format(save_dir,ftype), bbox_inches='tight')

---

## Energy resolution summary plot

In [ ]:
Qbb = 2458.

# Total errors entered by hand (includes systematics)
res_err_tot = [0.4, 0.1, 0.1, 0.2]
res_err_bb_tot = [0.1, 0.1, 0.1, 0.2]

# Tabulate values from fits to peaks.
mu_fit = np.array([mu_xray_fid, mu_cs_fid, mu_de_fid, mu_tl_fid])
mu_err = np.array([e_mu_xray_fid, e_mu_cs_fid, e_mu_de_fid, e_mu_tl_fid])
sigma_fit = np.array([sigma_xray_fid, sigma_cs_fid, sigma_de_fid, sigma_tl_fid])
sigma_err = np.array([e_sigma_xray_fid, e_sigma_cs_fid, e_sigma_de_fid, e_sigma_tl_fid])

# Calculate the resolution.
res     = 235. * (sigma_fit/mu_fit)
res_err = res * ((sigma_err/sigma_fit)**2 + (mu_err/mu_fit)**2)**0.5
res_bb = res * (mu_fit/Qbb)**0.5
res_bb_err = res_bb * ((res_err/res)**2 + (mu_err/mu_fit)**2/4)**0.5
res_ext = lambda x: res[0]*(mu_fit[0]/x)**0.5
E_ext = np.arange(mu_fit[0],mu_fit[-1],1)

# Fit to a + b*sqrt(E).
p, V = curve_fit(fres, mu_fit, res, sigma=res_err_tot, absolute_sigma=True)
a_res = p[0]
b_res = p[1]
a_err = V[0][0]**0.5
b_err = V[1][1]**0.5
Efit = np.arange(0,mu_fit[-1],1.)
Rfit = lambda x: a_res + b_res/x**0.5

fig = plt.figure()
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

#plt.plot(E_ext,res_ext(E_ext),'--',color='red',alpha=0.3,label='Ext. from 30 keV as 1/$\sqrt{E}$')
plt.errorbar(mu_fit,res,fmt='.',yerr=res_err_tot,markersize=8,color='red',label='Selected fiducial regions')
plt.errorbar(mu_fit,res_bb,fmt='.',yerr=res_err_bb_tot,markersize=8,color='blue',label='Selected fiducial regions, ext. to Q$_\\beta$$_\\beta$')
plt.plot(Efit,Rfit(Efit),'--',color='red',alpha=0.3,label="Fit: {:.2f} + {:.2f}/$\sqrt{{E}}$".format(a_res,b_res))
plt.plot(E_ext,np.ones(len(E_ext))*res_bb[0],linestyle='--',color='blue',alpha=0.3,label='Ext. from 30 keV to Q$_\\beta$$_\\beta$')

plt.ylim(0,6.)
plt.xlabel('Energy (keV)')
plt.ylabel('Resolution (%FWHM)')
plt.legend(loc=0)

plt.savefig("{}/CSTH_resolution_summary.pdf".format(save_dir), bbox_inches='tight')

# Print the results
print("--- WITH FIDUCIAL CUTS ---")
print("Fit means (keV) [xray, Cs, double-escape]:")
print(mu_fit)
print("Resolution (%FWHM) [xray, Cs, double-escape]:")
print(res)
print("Resolution (%FWHM) errors [xray, Cs, double-escape]")
print(res_err)
print("Resolution (%FWHM) at Qbb [xray, Cs, double-escape]:")
print(res_bb)
print("Resolution (%FWHM) at Qbb errors [xray, Cs, double-escape]")
print(res_bb_err)
print("Fit to a + b/sqrt(E): a = {} +/- {}, b = {} +/- {}".format(a_res,a_err,b_res,b_err))

# Save constant terms for later use.
res_const = res - res_ext(mu_fit)
print("Constant terms in resolution fit [xray, Cs, double-escape]")
print(res_const)

In [ ]:
Qbb = 2.458
sigeinv = 1/Qbb

#    exclude Kr data
newx = [.0297, .6616, 1.5947]
newy = [5.71, 1.45, 1.11]
newdy = [0.4, 0.1, 0.1]

new_einv = np.asarray([1./xx for xx in newx])
new_r2 = np.asarray([yy**2 for yy in newy])
new_dr2 = np.asarray([2.*yy*dy for yy,dy in zip(newy,newdy)])


####    fit the straght line 
p_res, V_res = curve_fit(flinear, newx, newy, sigma=newdy, absolute_sigma=True)
m_res = p_res[0]
b_res = p_res[1]


a = math.sqrt(slope_NEW)
a_err = math.sqrt(V_res[0][0])

c = math.sqrt(intercept_NEW - slope_NEW*sigeinv)
c_err = math.sqrt(V_res[1][1])

print('--------  resolution at Qbb  ',res_NEW, 'error   ', dres_NEW)
print('   slope a = ', a, ' +/- ', a_err)
print('   constant term c = ', c, ' +/- ', c_err)

leg = r'$\Delta$E/E(Q$_{{\beta\beta}})$ = (' 
leg = leg + "{:.2f}".format(res_NEW)+ r' $\pm$ ' +  "{:.2f}".format(dres_NEW) + ') % FWHM'

xfit = np.arange(0.1,new_einv[0],0.01)
pfit = np.poly1d([m_res,b_res])

fig = plt.figure()
fig.set_figheight(3.0)
fig.set_figwidth(12.0)

plt.plot(new_einv,new_r2,'bs',label=leg,ms=3.0)
plt.errorbar(new_einv,new_r2,yerr=new_dr2,fmt="bs",ms=0.1)
plt.plot(xfit, pfit(xfit), '-', color='red')
#plt.plot([0.0,new_einv[0]],[intercept_NEW - slope_NEW*sigeinv,(new_einv[0] - sigeinv)*slope_NEW+intercept_NEW],c='r',lw=1)

#plt.plot([sigeinv,sigeinv],[0,5],c='m',ls=':')
plt.axvline(sigeinv,linestyle=':',color='blue',linewidth=1.5)

leg = plt.legend()
leg.get_frame().set_edgecolor('black')
plt.grid(alpha=0.4)
plt.xlabel('1/E (MeV$^{-1}$)')
plt.ylabel(r'($\Delta$E/E, %FWHM)$^2$')
#plt.xscale('log')
#plt.yscale('log')

plt.savefig("{}/CSTH_resolution_extrapolation.pdf".format(save_dir), bbox_inches='tight')

---

## "S2 width" obtained from z

In [ ]:
# fit Cs for paper
rmax = 180
zmin = 0; zmax = 500
rcuts = C_basic & (A_rmax < rmax) & (A_zmin > zmin) & (A_zmax < zmax)

text_x = 184000*CAL_m + CAL_b
plt_rng = [180000*CAL_m + CAL_b,220000*CAL_m + CAL_b]
nbins = 120
nxybins = 25

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

fit_rng_low = 195300*CAL_m + CAL_b
fit_rng_high = 210900*CAL_m + CAL_b
fit_gauss_A = 2200 #95
fit_gauss_mu = 202600*CAL_m + CAL_b
fit_gauss_sigma = 4000*CAL_m + CAL_b
fit_exp_A = 50
fit_exp_mean = 3.0e4*CAL_m + CAL_b
fit_exp_x0 = 1.6e5*CAL_m + CAL_b

ax1 = fig.add_subplot(121)
y0, x0, _ = plt.hist(A_Ecplot[rcuts]*CAL_m + CAL_b, nbins, range=plt_rng, **hargs)

# Gaussian fit
x    = x0[:-1] + np.diff(x0) * 0.5
sel  = coref.in_range(x, fit_rng_low, fit_rng_high)
x, y = x[sel], y0[sel]
f    = fitf.fit(lambda x, *args: fitf.gauss(x, *args[:3]) + fitf.polynom(x, *args[3:]), x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, 10, -10, -10))
#f    = fitf.fit(gaussexpo, x, y, (fit_gauss_A, fit_gauss_mu, fit_gauss_sigma, fit_exp_A, fit_exp_mean, fit_exp_x0))
plt.plot(x, f.fn(x), "r")
plt.text(text_x, 3.5*max(y)/6, gausstext(f.values[0:3],E_Cs),fontsize=14)
plt.xlabel('E (keV)')
plt.ylabel('Counts/bin')
#plt.title("Rmax < {}, {} < Zmax < {}".format(rmax,zmin,zmax), fontsize=12, fontweight='bold')
print("Fit parameters for wide cuts (amp, mu, sigma, A, mean, x0)")
print(f.values)
print("Fit errors for wide cuts (amp, mu, sigma)")
print(f.errors)

xyz_rng_low = 650
xyz_rng_high = 675

ax4 = fig.add_subplot(122) 
h, x, y = plt.hist(A_zmax[rcuts & (A_Ecplot*CAL_m+CAL_b > xyz_rng_low) & (A_Ecplot*CAL_m+CAL_b < xyz_rng_high)] - A_zmin[rcuts & (A_Ecplot*CAL_m+CAL_b > xyz_rng_low) & (A_Ecplot*CAL_m+CAL_b < xyz_rng_high)],bins=50,range=[0,100],**hargs)
plt.xlabel("")
plt.ylabel("Counts/bin")
plt.title("zmax - zmin for $E\in ({},{})$".format(xyz_rng_low,xyz_rng_high), fontsize=14)

plt.savefig("{}/CSTH_zwidth.pdf".format(save_dir), bbox_inches='tight')

## Energy resolution summary

In [ ]:
Qbb = 2458.

# Total errors entered by hand (includes systematics)
res_err_tot = [0.4, 0.1, 0.1]
res_err_bb_tot = [0.1, 0.1, 0.1]

mu_fit = np.array([mu_xray_fid, mu_cs_fid, mu_de_fid])
mu_err = np.array([e_mu_xray_fid, e_mu_cs_fid, e_mu_de_fid])
sigma_fit = np.array([sigma_xray_fid, sigma_cs_fid, sigma_de_fid])
sigma_err = np.array([e_sigma_xray_fid, e_sigma_cs_fid, e_sigma_de_fid])

res     = 235. * (sigma_fit/mu_fit)
res_err = res * ((sigma_err/sigma_fit)**2 + (mu_err/mu_fit)**2)**0.5

res_bb = res * (mu_fit/Qbb)**0.5
res_bb_err = res_bb * ((res_err/res)**2 + (mu_err/mu_fit)**2/4)**0.5

res_ext = lambda x: res[0]*(mu_fit[0]/x)**0.5
E_ext = np.arange(mu_fit[0],mu_fit[-1],1)

fig = plt.figure()
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

plt.plot(E_ext,res_ext(E_ext),'--',color='red',alpha=0.3,label='Ext. from 30 keV as 1/$\sqrt{E}$')
plt.errorbar(mu_fit,res,fmt='.',yerr=res_err_tot,markersize=8,color='red',label='Selected fiducial regions')
plt.plot(E_ext,np.ones(len(E_ext))*res_bb[0],linestyle='--',color='blue',alpha=0.3,label='Ext. from 30 keV to Q$_\\beta$$_\\beta$')
plt.errorbar(mu_fit,res_bb,fmt='.',yerr=res_err_bb_tot,markersize=8,color='blue',label='Selected fiducial regions, ext. to Q$_\\beta$$_\\beta$')

plt.ylim(0,8.)
plt.xlabel('Energy (keV)')
plt.ylabel('Resolution (%FWHM)')
plt.legend(loc=0)

plt.savefig("{}/CSTH_energy_resolution.pdf".format(save_dir), bbox_inches='tight')

# Print the results
print("--- WITH FIDUCIAL CUTS ---")
print("Fit means (keV) [xray, Cs, double-escape]:")
print(mu_fit)
print("Resolution (%FWHM) [xray, Cs, double-escape]:")
print(res)
print("Resolution (%FWHM) errors [xray, Cs, double-escape]")
print(res_err)
print("Resolution (%FWHM) at Qbb [xray, Cs, double-escape]:")
print(res_bb)
print("Resolution (%FWHM) at Qbb errors [xray, Cs, double-escape]")
print(res_bb_err)

# Save constant terms for later use.
res_const = res - res_ext(mu_fit)
print("Constant terms in resolution fit [xray, Cs, double-escape]")
print(res_const)

## Characterize the number of voxels per event

In [ ]:
cuts_N_Cs = C_wide & ((A_Ec*CAL_m + CAL_b) > mu_cs_fid - 2*sigma_cs_fid) & ((A_Ec*CAL_m + CAL_b) < mu_cs_fid + 2*sigma_cs_fid)
cuts_N_Th = C_wide & ((A_Ec*CAL_m + CAL_b) > mu_de_fid - 2*sigma_de_fid) & ((A_Ec*CAL_m + CAL_b) < mu_de_fid + 2*sigma_de_fid)

nbins = 8

fig = plt.figure(1)
fig.set_figheight(4.0)
fig.set_figwidth(18.0)

# N for Cs
ax1 = fig.add_subplot(121)
hCs = plt.hist(A_nvox[cuts_N_Cs], bins=nbins, **hargs)
plt.xlabel("N (voxels/event)")
plt.ylabel("Counts/bin")
plt.title("Cs events")

# N for Th
ax2 = fig.add_subplot(122)
hz = plt.hist(A_nvox[cuts_N_Th], bins=nbins, **hargs)
plt.xlabel("N (voxels/event)")
plt.ylabel("Counts/bin")
plt.title("Th events")

plt.savefig("{}/CSTH_N_Cs_Th.pdf".format(save_dir), bbox_inches='tight', facecolor='#EEEEEC')

In [ ]:
# Profile plot
nxybins = 20
nbins = 20

fig = plt.figure(1)
fig.set_figheight(4.0)
fig.set_figwidth(18.0)

Nprof, Eprof, Eerr = fitf.profileX(A_nvox[C_wide],A_Ec[C_wide]*CAL_m+CAL_b,nbins=nbins,xrange=(0,100),yrange=(0,2500))
p, V = curve_fit(fsqrt, Nprof, Eprof)

h, x, y = np.histogram2d(-(A_Ec[C_wide]*CAL_m+CAL_b), A_nvox[C_wide], bins=[nxybins, nxybins], range=[[-3000,0],[0,50]])
#plt.scatter(A_nvox[C_wide], A_Ec[C_wide]*CAL_m+CAL_b)
plt.imshow(h, extent=[0,50,0,3000], interpolation = "gaussian", cmap='jet', aspect=1./100, norm=SymLogNorm(linthresh=1.0, linscale=1.0))
plt.plot(np.arange(2,50),fsqrt(np.arange(2,50),p[0],p[1]), linewidth=2, color='red', linestyle='dashed')
plt.xlabel("N (voxels/event)")
plt.ylabel("E (keV)")
plt.text(25, 250, "E(N) = {:.1f} + {:.1f}$\sqrt{{N}}$".format(p[1],p[0]), 
         bbox={'facecolor':'white', 'alpha':1.0, 'pad':5},fontsize=12)
#plt.errorbar(Nprof,Eprof,yerr=Eerr)

In [ ]:
plt.plot(mu_fit,res_const,'.',color='black')
plt.plot(mu_fit,res_const,'-',color='black')
plt.xlabel("Energy (keV)")
plt.ylabel("Constant resolution term A")

---

---

# Plot waveforms
Note that for run 4735, the following events were found to have been located within 1 sigma of the double-escape peak (with wide cuts):
```
    72,    149,    636,   1708,   1877,   1916,   2167,   2474,
  2990,   3110,   3113,   3588,   4063,   5206,   5307,   5719,
  7260,   7331,   7386,   8367,   8617,   8868,   9404,   9920,
  ...
```

In [ ]:
# configuration parameters
DataPMT = load_db.DataPMT(4735)

coeff_c = DataPMT.coeff_c.values
coeff_blr = DataPMT.coeff_blr.values
pmt_active = np.nonzero(DataPMT.Active.values.astype(bool))[0].tolist()
n_baseline = 78000
thr_trigger = 5
accum_discharge_length = 5000

In [ ]:
wffile = '/Users/jrenner/local/data/NEW/4735/data/dst_waves.gdcsnext.001_4735.root.h5'
event_numbers, timestamps = tbl.get_event_numbers_and_timestamps_from_file_name(wffile)
h5in = tb.open_file(wffile, "r")
NEVT_pmt, pmtrwf, sipmrwf, pmtblr = tbl.get_rwf_vectors(h5in)

In [ ]:
# select the waveform and deconvolve
evt_wf = 149
iwf = -1
for ii,ee in enumerate(event_numbers):
    if(ee == evt_wf):
        if(iwf > 0): print("WARNING: selected event twice!")
        iwf = ii

RWF = pmtrwf[iwf]
CWF = blr.deconv_pmt(RWF,coeff_c,coeff_blr,pmt_active,n_baseline,thr_trigger,accum_discharge_length)

sum_RWF = np.sum(RWF,axis=0)
sum_CWF = np.sum(CWF,axis=0)

print("Selected event number {} at index {}".format(evt_wf,iwf))

In [ ]:
# plot the corrected and uncorrected waveforms
fig = plt.figure(3)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(211)
ax1.plot(sum_RWF,color='black')
ax1.set_xlim(30000,50000)
#ax1.set_xlabel('Sample')
ax1.set_ylabel('ADC counts')
ax1.text(45700, 5.0*max(sum_RWF)/8, 'Uncorrected waveform',fontsize=14) #bbox={'facecolor':'white', 'alpha':1.0, 'pad':5}

ax2 = fig.add_subplot(212)
ax2.plot(sum_CWF,color='black')
ax2.set_xlim(30000,50000)
ax2.set_xlabel('Sample')
ax2.set_ylabel('ADC counts')
ax2.text(46100, 2.6*max(sum_RWF)/8, 'Corrected waveform',fontsize=14) #bbox={'facecolor':'white', 'alpha':1.0, 'pad':5}

plt.savefig("{}/CSTH_WF_run4735_evt{}.pdf".format(save_dir,evt_wf), bbox_inches='tight')

In [ ]:
# plot the corrected waveform
fig = plt.figure()
fig.set_figheight(3.0)
fig.set_figwidth(12.5)

WFSCALE = 25./1000
wf_time = np.arange(0,len(sum_CWF),1.)*WFSCALE

ax1 = fig.add_subplot(111)
ax1.plot(wf_time,sum_CWF,'-',color='black')
ax1.set_xlim(32500*WFSCALE,45500*WFSCALE)
plt.annotate('', xy=(35050*WFSCALE,2500),  xycoords='data',
                xytext=(34050*WFSCALE,5600), textcoords='data',
                arrowprops=dict(arrowstyle="->",facecolor='black'))
#                arrowprops=dict(arrowstyle="-[,widthB=1"))
#ax1.locator_params(axis='y', nbins=5)
#ax1.axis([0,130,654,670])
ax1.set_xlabel('Time ($\mu$s)')
ax1.set_ylabel('ADC counts')

#ax2 = ax1.twinx()
#ax2.set_ylim(ax1.get_ylim())
#y2ticks = np.array([4000, 2000, 0]) #, 350, 400])
#ax2.set_yticks(y2ticks)
#ax2.set_yticklabels(y2ticks) #,fontsize=tc_tlabel_size,fontname=tc_tlabel_font,fontweight=tc_tlabel_weight)
#ax2.set_ylabel('ADC counts')

# S1 inset
ax1i = fig.add_axes([0.18,0.60,0.15,0.2])
ax1i.plot(wf_time,sum_CWF,'-',color='black')
ax1i.axis([35000*WFSCALE,35100*WFSCALE,0,2500])
ax1i.xaxis.set_ticks([35000*WFSCALE,35050*WFSCALE,35100*WFSCALE])
ax1i.yaxis.set_ticks([0,2500])
#ax2i = ax1i.twinx()
#ax2i.set_ylim(ax1i.get_ylim())
#y2iticks = np.array([160, 80, 0]) #, 350, 400])
#ax2i.set_yticks(y2iticks)
#ax2i.set_yticklabels(y2iticks) #,fontsize=tc_tlabel_size,fontname=tc_tlabel_font,fontweight=tc_tlabel_weight)
        
# S1 and S2 labels
fig.text(0.30,0.72,"S1",color='black')
fig.text(0.685,0.18,"S2",color='black')
#fig.text(0.26,0.25,"NaI",color='black')
#formatPlot(plt)
lb_x = ax1i.xaxis.get_majorticklabels()
lb_y = ax1i.yaxis.get_majorticklabels()
#for lb in (lb_x + lb_y):
#    lb.set_fontname(tc_tlabel_font)
#    lb.set_fontweight(tc_tlabel_weight)
#    lb.set_fontsize(tc_tlabel_size*0.85)

plt.savefig("{}/CSTH_WF_run4735_evt{}.pdf".format(save_dir,evt_wf), bbox_inches='tight')

-------

# Plot voxelized tracks

In [ ]:
vox_fname = "/Users/jrenner/local/data/NEW/4735/15x15x15/voxels_combined_4735.h5"
vox_size = [15., 15., 15.]
vol_min = [0., 0., 0.]
vol_max = [450., 450., 450.]
h5f = h5py.File(vox_fname,'r')

In [ ]:
ntrk = 8
trks = [hk for hk in h5f.keys()]
trk_name = "run4735"
emin = 0.
emax = 160.

# Read in the track.
trkmat = h5f[trks[ntrk]]
varr_x = trkmat[0]*vox_size[0]
varr_y = trkmat[1]*vox_size[1]
varr_z = trkmat[2]*vox_size[2]
varr_c = trkmat[3]*CAL_m + CAL_b
print("Total E = {}".format(sum(varr_c)*CAL_m + CAL_b))

# plot the voxelized tracks
# 2D histogram
fig = plt.figure();
fig.set_figheight(3.0);
fig.set_figwidth(12.0);
    
vtrk_max = 1.0*np.array([np.max(varr_x),np.max(varr_y),np.max(varr_z)])
vtrk_min = 1.0*np.array([np.min(varr_x),np.min(varr_y),np.min(varr_z)])
vtrk_len = np.abs(vtrk_max-vtrk_min)

vtrk_max = vtrk_max + 0.5*vtrk_len
vtrk_min = vtrk_min - 0.5*vtrk_len

vtrk_len = np.abs(vtrk_max-vtrk_min)
vtrk_maxlen = np.max(vtrk_len)
vtrk_max = vtrk_max + (vtrk_maxlen - vtrk_len)/2
vtrk_min = vtrk_min - (vtrk_maxlen - vtrk_len)/2
print("track min = {}".format(vtrk_min))
print("track max = {}".format(vtrk_max))
print("energy min = {}, energy max = {}".format(min(varr_c),max(varr_c)))

gs = gridspec.GridSpec(1, 5, width_ratios=[10, 1, 10, 1, 10]) 

# create the x-y projection
ax1 = fig.add_subplot(gs[0])
hxy, xxy, yxy = np.histogram2d(varr_y, varr_x, weights=varr_c, normed=False, bins=(1.0*(vol_max[1]-vol_min[1])/vox_size[1], 1.0*(vol_max[0]-vol_min[0])/vox_size[0]), range=[[vol_min[1],vol_max[1]],[vol_min[0],vol_max[0]]])
extent1 = [yxy[0], yxy[-1], xxy[0], xxy[-1]]
sp1 = ax1.imshow(hxy, extent=extent1, interpolation='none', aspect='auto', origin='lower', cmap='jet', vmin=emin, vmax=emax)
ax1.set_xlabel("x (mm)")
ax1.set_ylabel("y (mm)")
ax1.set_xlim([vtrk_min[0],vtrk_max[0]])
ax1.set_ylim([vtrk_min[1],vtrk_max[1]])
#cbp1 = plt.colorbar(sp1)
#cbp1.set_label('Energy (keV)')

# Create the y-z projection.
ax2 = fig.add_subplot(gs[2])
hyz, xyz, yyz = np.histogram2d(varr_z, varr_y, weights=varr_c, normed=False, bins=(1.0*(vol_max[2]-vol_min[2])/vox_size[2], 1.0*(vol_max[1]-vol_min[1])/vox_size[1]), range=[[vol_min[2],vol_max[2]],[vol_min[1],vol_max[1]]])
extent2 = [yyz[0], yyz[-1], xyz[0], xyz[-1]]
sp2 = ax2.imshow(hyz, extent=extent2, interpolation='none', aspect='auto', origin='lower', cmap='jet', vmin=emin, vmax=emax)
ax2.set_xlabel("y (mm)")
ax2.set_ylabel("z (mm)")
ax2.set_xlim([vtrk_min[1],vtrk_max[1]])
ax2.set_ylim([vtrk_min[2],vtrk_max[2]])
#cbp2 = plt.colorbar(sp2)
#cbp2.set_label('Energy (keV)')

# Create the x-z projection.
ax3 = fig.add_subplot(gs[4])
hxz, xxz, yxz = np.histogram2d(varr_z, varr_x, weights=varr_c, normed=False, bins=(1.0*(vol_max[2]-vol_min[2])/vox_size[2], 1.0*(vol_max[0]-vol_min[0])/vox_size[0]), range=[[vol_min[2],vol_max[2]],[vol_min[0],vol_max[0]]])
extent3 = [yxz[0], yxz[-1], xxz[0], xxz[-1]]
sp3 = ax3.imshow(hxz, extent=extent3, interpolation='none', aspect='auto', origin='lower', cmap='jet', vmin=emin, vmax=emax)
ax3.set_xlabel("x (mm)")
ax3.set_ylabel("z (mm)")
ax3.set_xlim([vtrk_min[0],vtrk_max[0]])
ax3.set_ylim([vtrk_min[2],vtrk_max[2]])
#cbp3 = plt.colorbar(sp3)

cax = fig.add_axes([0.27, -0.15, 0.45, 0.05])
cbp3 = plt.colorbar(sp3, cax=cax, orientation='horizontal')
cbp3.set_label('Energy (keV)')

plt.savefig("{}/CSTH_{}_{}.png".format(save_dir,trk_name,trks[ntrk]), bbox_inches='tight')

---

## Histogram individual track lengths for different energy ranges

In [ ]:
nbins = 45

# Get the track lengths from the present data.
lmtrk_cs = A_lmtrk[C_wide & (A_ntrks == 1) & ((A_Ec*CAL_m + CAL_b) > (mu_cs_fid - 3*sigma_cs_fid)) & ((A_Ec*CAL_m + CAL_b) < (mu_cs_fid + 3*sigma_cs_fid))]
lmtrk_de = A_lmtrk[C_wide & (A_ntrks == 1) & ((A_Ec*CAL_m + CAL_b) > (mu_de_fid - 3*sigma_de_fid)) & ((A_Ec*CAL_m + CAL_b) < (mu_de_fid + 3*sigma_de_fid))]
#lmtrk_tl = A_lmtrk[(A_ntrks == 1) & ((A_Ec*CAL_m + CAL_b) > (mu_tl_fid - 2*sigma_tl_fid)) & ((A_Ec*CAL_m + CAL_b) < (mu_tl_fid + 2*sigma_tl_fid))] 
print("Cs photopeak energy range: [{:.2f},{:.2f}] keV".format((mu_cs_fid - 2*sigma_cs_fid), (mu_cs_fid + 2*sigma_cs_fid)))
print("e+e- event energy range: [{:.2f},{:.2f}] keV".format((mu_de_fid - 2*sigma_de_fid), (mu_de_fid + 2*sigma_de_fid)))
#print("Tl photopeak energy range: [{:.2f},{:.2f}] keV".format((mu_tl_fid - 2*sigma_tl_fid), (mu_tl_fid + 2*sigma_tl_fid)))

# Get the track lengths from the MC simulations (0vbb at 15 bar, 15x15x15 voxelization).
ftl_MC = np.loadtxt("/Users/jrenner/local/data/NEW/trk_len_0vbb_15bar_15x15x15.dat")
lmtrk_MC = ftl_MC[:,1]

fig = plt.figure(1)
fig.set_figheight(4.0)
fig.set_figwidth(12.0)

gs = gridspec.GridSpec(1, 3, width_ratios=[10, 1, 10]) 

#ax1 = fig.add_subplot(gs[0])
#y, x, _ = plt.hist(lmtrk_cs, nbins, **hargs)
#ax1.set_xlabel("Track length (mm)")
#ax1.set_ylabel("Counts/bin")
#plt.text(70, 5.5*max(y)/6, "$^{137}$Cs photopeak",fontsize=18)
#print("Cs track length = {} +/- {}".format(lmtrk_cs.mean(),lmtrk_cs.std()))

ax1 = fig.add_subplot(121)
y, x, _ = plt.hist(lmtrk_de, nbins, **hargs)
ax1.set_xlabel("Track length (mm)")
ax1.set_ylabel("Counts/bin")
plt.text(270, 4.8*max(y)/6, "e$^{{+}}$e$^{{-}}$ events (DATA, 7.2 bar)\n$\mu = {:.1f}$ mm\n$\sigma = {:.1f}$ mm".format(lmtrk_de.mean(), lmtrk_de.std()),fontsize=12,ha='right')
print("Double escape track length = {} +/- {}".format(lmtrk_de.mean(),lmtrk_de.std()))

ax2 = fig.add_subplot(122)
y, x, _ = plt.hist(lmtrk_MC, nbins, **hargs)
ax2.set_xlabel("Track length (mm)")
ax2.set_ylabel("Counts/bin")
plt.text(215, 4.8*max(y)/6, "0$\\nu\\beta\\beta$ events (MC, 15 bar)\n$\mu = {:.1f}$ mm\n$\sigma = {:.1f}$ mm".format(lmtrk_MC.mean(), lmtrk_MC.std()),fontsize=12,ha='right')
print("0vbb (15 bar) track length = {} +/- {}".format(lmtrk_MC.mean(),lmtrk_MC.std()))

plt.savefig("{}/CSTH_trk_lengths.pdf".format(save_dir), bbox_inches='tight')

## Energy vs. number of hits

In [ ]:
# Energy vs. # of hits
cuts_T = C_wide

nbins_prof = 8
nbins_E = 30

# for double-escape peak
Thist_min_D = 33
Thist_max_D = 52
Ehist_min_D = 1570
Ehist_max_D = 1640
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1670

# for Cs peak
Thist_min_Cs = 12
Thist_max_Cs = 21
Ehist_min_Cs = 648
Ehist_max_Cs = 675
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 640
plt_rng_Ehigh_Cs = 680

Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(13.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(221);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_nhits[cuts_T], bins=[nbins_E, Thist_max_D - Thist_min_D], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax1.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.title("Energy vs. # of hits")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('# of hits')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(222);
ax2.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax2.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. # of hits profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('# of hits')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(223);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_nhits[cuts_T], bins=[nbins_E, Thist_max_Cs - Thist_min_Cs], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax5.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.title("Energy vs. # of hits")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('# of hits')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(224);
ax6.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax6.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. # of hits profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('# of hits')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_nhits.{}".format(save_dir,ftype), bbox_inches='tight')

## Energy vs. track length

In [ ]:
# Energy vs. time
cuts_T = C_wide & (A_ntrks == 1)

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for double-escape peak
Thist_min_D = 40
Thist_max_D = 160
Ehist_min_D = 1570
Ehist_max_D = 1640
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1670

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 60
Ehist_min_Cs = 648
Ehist_max_Cs = 675
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 640
plt_rng_Ehigh_Cs = 680

Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_lmtrk[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_lmtrk[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(13.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(221);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_lmtrk[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax1.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. track length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('track length (mm)')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(222);
ax2.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax2.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. track length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('track length (mm)')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(223);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_lmtrk[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax5.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. track length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('track length (mm)')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(224);
ax6.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax6.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. track length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('track length (mm)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_ltrk.{}".format(save_dir,ftype), bbox_inches='tight')

In [ ]:
# Energy vs. z-length
cuts_T = C_wide

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 34
Ehist_min_Cs = 650
Ehist_max_Cs = 670
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 640
plt_rng_Ehigh_Cs = 680

# for double-escape peak
Thist_min_D = 24
Thist_max_D = 72
Ehist_min_D = 1580
Ehist_max_D = 1630
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1670

# for Tl peak
Thist_min_Tl = 40
Thist_max_Tl = 150
Ehist_min_Tl = 2580
Ehist_max_Tl = 2690
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = 2550
plt_rng_Ehigh_Tl = 2700

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(flinear, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],10.)
pfit_Cs = np.poly1d([m_Cs,b_Cs])
print("Slope for correction, Cs peak: {}".format(m_Cs))

p_D, V_D = curve_fit(flinear, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],10.)
pfit_D = np.poly1d([m_D,b_D])
print("Slope for correction, double-escape peak: {}".format(m_D))

p_Tl, V_Tl = curve_fit(flinear, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],10.)
pfit_Tl = np.poly1d([m_Tl,b_Tl])
print("Slope for correction, Tl peak: {}".format(m_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m = {:.3f} keV/mm".format(m_Cs))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m = {:.3f} keV/mm".format(m_D))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m = {:.3f} keV/mm".format(m_Tl))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_zlen.{}".format(save_dir,"png"), bbox_inches='tight')

In [ ]:
# Energy vs. r-length
A_xlen = (A_xmax - A_xmin)
A_ylen = (A_ymax - A_ymin)
A_rlen = (A_xlen**2 + A_ylen**2)**0.5
cuts_T = C_wide

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for Cs peak
Thist_min_Cs = 25
Thist_max_Cs = 70
Ehist_min_Cs = 650
Ehist_max_Cs = 670
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 640
plt_rng_Ehigh_Cs = 680

# for double-escape peak
Thist_min_D = 55
Thist_max_D = 110
Ehist_min_D = 1580
Ehist_max_D = 1630
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1670

# for Tl peak
Thist_min_Tl = 100
Thist_max_Tl = 240
Ehist_min_Tl = 2580
Ehist_max_Tl = 2690
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = 2550
plt_rng_Ehigh_Tl = 2700

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(A_rlen[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_rlen[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_rlen[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(flinear, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],1.)
pfit_Cs = np.poly1d([m_Cs,b_Cs])
print("Slope for correction, Cs peak: {}".format(m_Cs))

p_D, V_D = curve_fit(flinear, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],1.)
pfit_D = np.poly1d([m_D,b_D])
print("Slope for correction, double-escape peak: {}".format(m_D))

p_Tl, V_Tl = curve_fit(flinear, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],1.)
pfit_Tl = np.poly1d([m_Tl,b_Tl])
print("Slope for correction, Tl peak: {}".format(m_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_rlen[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. r-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('r-length (mm)')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m = {:.3f} keV/mm".format(m_Cs))
plt.legend(loc=2, fontsize=14)
plt.title("Energy vs. r-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('r-length (mm)')
plt.ylabel('E (keV)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_rlen[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. r-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('r-length (mm)')
plt.ylabel('E (keV)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m = {:.3f} keV/mm".format(m_D))
plt.legend(loc=2, fontsize=14)
plt.title("Energy vs. r-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('r-length (mm)')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_rlen[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. r-length")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('r-length (mm)')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m = {:.3f} keV/mm".format(m_Tl))
plt.legend(loc=2, fontsize=14)
plt.title("Energy vs. r-length profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('r-length (mm)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_rlen.{}".format(save_dir,"png"), bbox_inches='tight')

In [ ]:
# S1 vs. z-length
cuts_T = C_wide

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 34
Ehist_min_Cs = 130
Ehist_max_Cs = 205
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 80
plt_rng_Ehigh_Cs = 250
ES2_min_Cs = 650
ES2_max_Cs = 670

# for double-escape peak
Thist_min_D = 24
Thist_max_D = 72
Ehist_min_D = 280
Ehist_max_D = 570
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 300
plt_rng_Ehigh_D = 550
ES2_min_D = 1580
ES2_max_D = 1630

# for Tl peak
Thist_min_Tl = 40
Thist_max_Tl = 150
Ehist_min_Tl = 530
Ehist_max_Tl = 870
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = 550
plt_rng_Ehigh_Tl = 870
ES2_min_Tl = 2580
ES2_max_Tl = 2690

# Modified cuts (to include S2 ranges)
cuts_T_Cs = cuts_T & ((A_Ec*CAL_m + CAL_b) > ES2_min_Cs) & ((A_Ec*CAL_m + CAL_b) < ES2_max_Cs)
cuts_T_D  = cuts_T & ((A_Ec*CAL_m + CAL_b) > ES2_min_D) & ((A_Ec*CAL_m + CAL_b) < ES2_max_D)
cuts_T_Tl = cuts_T & ((A_Ec*CAL_m + CAL_b) > ES2_min_Tl) & ((A_Ec*CAL_m + CAL_b) < ES2_max_Tl)

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(A_zmax[cuts_T_Tl] - A_zmin[cuts_T_Tl],A_ES1[cuts_T_Tl],nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_zmax[cuts_T_D] - A_zmin[cuts_T_D],A_ES1[cuts_T_D],nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_zmax[cuts_T_Cs] - A_zmin[cuts_T_Cs],A_ES1[cuts_T_Cs],nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(flinear, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],10.)
pfit_Cs = np.poly1d([m_Cs,b_Cs])
print("Slope for correction, Cs peak: {}".format(m_Cs))

p_D, V_D = curve_fit(flinear, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],10.)
pfit_D = np.poly1d([m_D,b_D])
print("Slope for correction, double-escape peak: {}".format(m_D))

p_Tl, V_Tl = curve_fit(flinear, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],10.)
pfit_Tl = np.poly1d([m_Tl,b_Tl])
print("Slope for correction, Tl peak: {}".format(m_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-A_ES1[cuts_T_Cs], A_zmax[cuts_T_Cs] - A_zmin[cuts_T_Cs], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("S1 vs. z-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('z-length (mm)')
plt.ylabel('S1 (pe)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m = {:.3f} pe/mm".format(m_Cs))
plt.legend(loc=1, fontsize=14)
plt.title("S1 vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('z-length (mm)')
plt.ylabel('S1 (pe)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-A_ES1[cuts_T_D], A_zmax[cuts_T_D] - A_zmin[cuts_T_D], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("S1 vs. z-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('z-length (mm)')
plt.ylabel('S1 (pe)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m = {:.3f} pe/mm".format(m_D))
plt.legend(loc=1, fontsize=14)
plt.title("S1 vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('z-length (mm)')
plt.ylabel('S1 (pe)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-A_ES1[cuts_T_Tl], A_zmax[cuts_T_Tl] - A_zmin[cuts_T_Tl], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("S1 vs. z-length")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('z-length (mm)')
plt.ylabel('S1 (pe)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m = {:.3f} pe/mm".format(m_Tl))
plt.legend(loc=1, fontsize=14)
plt.title("S1 vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('z-length (mm)')
plt.ylabel('S1 (pe)')

plt.savefig("{}/CSTH_S1_vs_zlen.{}".format(save_dir,"png"), bbox_inches='tight')

In [ ]:
# Energy vs. angle with z-axis
A_xlen = (A_xmax - A_xmin)
A_ylen = (A_ymax - A_ymin)
A_zlen = (A_zmax - A_zmin)
A_trklen = (A_xlen**2 + A_ylen**2 + A_zlen**2)**0.5
cuts_T = C_wide & (A_ntrks == 1) & (A_trklen > 0)

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

radtodeg = (180/np.pi)

# for Cs peak
Thist_min_Cs = 0.9*radtodeg
Thist_max_Cs = 1.4*radtodeg
Ehist_min_Cs = 650
Ehist_max_Cs = 670
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 645
plt_rng_Ehigh_Cs = 675

# for double-escape peak
Thist_min_D = 0.7*radtodeg
Thist_max_D = 1.4*radtodeg
Ehist_min_D = 1570
Ehist_max_D = 1630
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1650

# for Tl peak
Thist_min_Tl = 0.7*radtodeg
Thist_max_Tl = 1.4*radtodeg
Ehist_min_Tl = 2570
Ehist_max_Tl = 2680
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = 2550
plt_rng_Ehigh_Tl = 2700

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(radtodeg*np.arccos(A_zlen[cuts_T]/A_trklen[cuts_T]),A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(radtodeg*np.arccos(A_zlen[cuts_T]/A_trklen[cuts_T]),A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(radtodeg*np.arccos(A_zlen[cuts_T]/A_trklen[cuts_T]),A_Ec[cuts_T]*CAL_m+CAL_b,nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(flinear, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],1.)
pfit_Cs = np.poly1d([m_Cs,b_Cs])
print("Slope for correction, Cs peak: {}; intercept: {}".format(m_Cs, b_Cs))

p_D, V_D = curve_fit(flinear, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],1.)
pfit_D = np.poly1d([m_D,b_D])
print("Slope for correction, double-escape peak: {}".format(m_D))

p_Tl, V_Tl = curve_fit(flinear, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],1.)
pfit_Tl = np.poly1d([m_Tl,b_Tl])
print("Slope for correction, Tl peak: {}".format(m_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), radtodeg*np.arccos(A_zlen[cuts_T]/A_trklen[cuts_T]), bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-angle $\\theta$")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('$\\theta$ (degrees)')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m = {:.3f} keV/degree".format(m_Cs))
plt.legend(loc=2, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('$\\theta$ (degrees)')
plt.ylabel('E (keV)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), radtodeg*np.arccos(A_zlen[cuts_T]/A_trklen[cuts_T]), bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-angle $\\theta$")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('$\\theta$ (degrees)')
plt.ylabel('E (keV)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m = {:.3f} keV/degree".format(m_D))
plt.legend(loc=2, fontsize=14)
plt.title("Energy vs. z-angle profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('$\\theta$ (degrees)')
plt.ylabel('E (keV)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), radtodeg*np.arccos(A_zlen[cuts_T]/A_trklen[cuts_T]), bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-angle $\\theta$")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('$\\theta$ (degrees)')
plt.ylabel('E (keV)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m = {:.3f} keV/degree".format(m_Tl))
plt.legend(loc=2, fontsize=14)
plt.title("Energy vs. z-angle profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('$\\theta$ (degrees)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_zlen.{}".format(save_dir,"png"), bbox_inches='tight')

## Energy vs. z-length, with correction

In [ ]:
# Energy vs. z-length
cuts_T = C_wide

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for double-escape peak
Thist_min_D = 24
Thist_max_D = 72
Ehist_min_D = 1580
Ehist_max_D = 1630
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = 1550
plt_rng_Ehigh_D = 1670

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 34
Ehist_min_Cs = 650
Ehist_max_Cs = 670
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = 640
plt_rng_Ehigh_Cs = 680

Tprof_D_corr, Eprof_D_corr, Eerr_D_corr = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],1605*(A_Ec[cuts_T]*CAL_m+CAL_b)/prof_D(A_zmax[cuts_T] - A_zmin[cuts_T]),nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs_corr, Eprof_Cs_corr, Eerr_Cs_corr = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],660*(A_Ec[cuts_T]*CAL_m+CAL_b)/prof_Cs(A_zmax[cuts_T] - A_zmin[cuts_T]),nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_D_corr  = interp1d(Tprof_D_corr, Eprof_D_corr, fill_value="extrapolate", kind="cubic")
prof_Cs_corr = interp1d(Tprof_Cs_corr, Eprof_Cs_corr, fill_value="extrapolate", kind="cubic")

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(13.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(221);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-1600*(A_Ec[cuts_T]*CAL_m + CAL_b)/prof_D(A_zmax[cuts_T] - A_zmin[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax1.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax2 = fig.add_subplot(222);
ax2.errorbar(Tprof_D_corr,Eprof_D_corr,yerr=Eerr_D)
ax2.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D_corr(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax3 = fig.add_subplot(223);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ec[cuts_T]*CAL_m + CAL_b), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax3.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

ax4 = fig.add_subplot(224);
ax4.errorbar(Tprof_Cs_corr,Eprof_Cs_corr,yerr=Eerr_Cs)
ax4.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs_corr(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('z-length (mm)')
plt.ylabel('E (keV)')

plt.savefig("{}/CSTH_E_vs_zlen_corr.{}".format(save_dir,"png"), bbox_inches='tight')

In [ ]:
# Energy vs. z-length (in SPE)
cuts_T = C_wide & (A_ntrks == 1)

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 34
Ehist_min_Cs = (650 - CAL_b)/CAL_m
Ehist_max_Cs = (668 - CAL_b)/CAL_m
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = (640 - CAL_b)/CAL_m
plt_rng_Ehigh_Cs = (680 - CAL_b)/CAL_m

# for double-escape peak
Thist_min_D = 24
Thist_max_D = 72
Ehist_min_D = (1570 - CAL_b)/CAL_m
Ehist_max_D = (1630 - CAL_b)/CAL_m
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = (1550 - CAL_b)/CAL_m
plt_rng_Ehigh_D = (1670 - CAL_b)/CAL_m

# for Tl peak
Thist_min_Tl = 40
Thist_max_Tl = 150
Ehist_min_Tl = (2570 - CAL_b)/CAL_m
Ehist_max_Tl = (2680 - CAL_b)/CAL_m
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = (2550 - CAL_b)/CAL_m
plt_rng_Ehigh_Tl = (2700 - CAL_b)/CAL_m

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(fquadratic, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],1.)
pfit_Cs = np.poly1d([m_Cs, 0, b_Cs])
print("Cs peak, slope for correction: {}; intercept: {}".format(m_Cs, b_Cs))

p_D, V_D = curve_fit(fquadratic, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],1.)
pfit_D = np.poly1d([m_D, 0, b_D])
print("Double-escape peak: slope for correction {}; intercept: {}".format(m_D, b_D))

p_Tl, V_Tl = curve_fit(fquadratic, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],1.)
pfit_Tl = np.poly1d([m_Tl, 0, b_Tl])
print("Tl peak: slope for correction {}; intercept: {}".format(m_Tl, b_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ecplot[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m = {:.4f} PE/mm$^2$".format(m_Cs))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ecplot[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m = {:.4f} PE/mm$^2$".format(m_D))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ecplot[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m = {:.4f} PE/mm$^2$".format(m_Tl))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

plt.savefig("{}/CSTH_E_vs_zlen_PE.{}".format(save_dir,"png"), bbox_inches='tight')

In [ ]:
# Energy vs. z-length (in SPE), linear fit
cuts_T = C_wide & (A_ntrks == 1)

nbins_prof = 8
nbins_E = 30
nbins_ltrk = 10

# for Cs peak
Thist_min_Cs = 10
Thist_max_Cs = 34
Ehist_min_Cs = (650 - CAL_b)/CAL_m
Ehist_max_Cs = (668 - CAL_b)/CAL_m
plt_rng_Tlow_Cs = Thist_min_Cs
plt_rng_Thigh_Cs = Thist_max_Cs
plt_rng_Elow_Cs = (640 - CAL_b)/CAL_m
plt_rng_Ehigh_Cs = (680 - CAL_b)/CAL_m

# for double-escape peak
Thist_min_D = 24
Thist_max_D = 72
Ehist_min_D = (1570 - CAL_b)/CAL_m
Ehist_max_D = (1630 - CAL_b)/CAL_m
plt_rng_Tlow_D = Thist_min_D
plt_rng_Thigh_D = Thist_max_D
plt_rng_Elow_D = (1550 - CAL_b)/CAL_m
plt_rng_Ehigh_D = (1670 - CAL_b)/CAL_m

# for Tl peak
Thist_min_Tl = 40
Thist_max_Tl = 150
Ehist_min_Tl = (2570 - CAL_b)/CAL_m
Ehist_max_Tl = (2680 - CAL_b)/CAL_m
plt_rng_Tlow_Tl = Thist_min_Tl
plt_rng_Thigh_Tl = Thist_max_Tl
plt_rng_Elow_Tl = (2550 - CAL_b)/CAL_m
plt_rng_Ehigh_Tl = (2700 - CAL_b)/CAL_m

Tprof_Tl, Eprof_Tl, Eerr_Tl = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_Tl,Thist_max_Tl),yrange=(Ehist_min_Tl,Ehist_max_Tl))
Tprof_D, Eprof_D, Eerr_D = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_D,Thist_max_D),yrange=(Ehist_min_D,Ehist_max_D))
Tprof_Cs, Eprof_Cs, Eerr_Cs = fitf.profileX(A_zmax[cuts_T] - A_zmin[cuts_T],A_Ecplot[cuts_T],nbins=nbins_prof,xrange=(Thist_min_Cs,Thist_max_Cs),yrange=(Ehist_min_Cs,Ehist_max_Cs))

prof_Tl  = interp1d(Tprof_Tl, Eprof_Tl, fill_value="extrapolate", kind="cubic")
prof_D  = interp1d(Tprof_D, Eprof_D, fill_value="extrapolate", kind="cubic")
prof_Cs = interp1d(Tprof_Cs, Eprof_Cs, fill_value="extrapolate", kind="cubic")

# Fit to determine the correction functions.
p_Cs, V_Cs = curve_fit(flinear, Tprof_Cs, Eprof_Cs, sigma=Eerr_Cs, absolute_sigma=True)
m_Cs = p_Cs[0]
b_Cs = p_Cs[1]
sigma_m_Cs = V_Cs[0,0]**0.5
sigma_b_Cs = V_Cs[1,1]**0.5
sigma2_mb_Cs = V_Cs[0,1]
xfit_Cs = np.arange(Tprof_Cs[0],Tprof_Cs[-1],1.)
pfit_Cs = np.poly1d([m_Cs, b_Cs])
print("Cs peak, slope for correction: {}; intercept: {};  m/b = {}".format(m_Cs, b_Cs, m_Cs/b_Cs))

p_D, V_D = curve_fit(flinear, Tprof_D, Eprof_D, sigma=Eerr_D, absolute_sigma=True)
m_D = p_D[0]
b_D = p_D[1]
sigma_m_D = V_D[0,0]**0.5
sigma_b_D = V_D[1,1]**0.5
sigma2_mb_D = V_D[0,1]
xfit_D = np.arange(Tprof_D[0],Tprof_D[-1],1.)
pfit_D = np.poly1d([m_D, b_D])
print("Double-escape peak: slope for correction {}; intercept: {};  m/b = {}".format(m_D, b_D, m_D/b_D))

p_Tl, V_Tl = curve_fit(flinear, Tprof_Tl, Eprof_Tl, sigma=Eerr_Tl, absolute_sigma=True)
m_Tl = p_Tl[0]
b_Tl = p_Tl[1]
sigma_m_Tl = V_Tl[0,0]**0.5
sigma_b_Tl = V_Tl[1,1]**0.5
sigma2_mb_Tl = V_Tl[0,1]
xfit_Tl = np.arange(Tprof_Tl[0],Tprof_Tl[-1],1.)
pfit_Tl = np.poly1d([m_Tl, b_Tl])
print("Tl peak: slope for correction {}; intercept: {}; m/b = {}".format(m_Tl, b_Tl, m_Tl/b_Tl))

fig = plt.figure()
fig.patch.set_alpha(0.0)
fig.set_figheight(18.0)
fig.set_figwidth(18.0)

ax1 = fig.add_subplot(321);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ecplot[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Cs,-plt_rng_Elow_Cs],[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs]])
plt.imshow(h, extent=[plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,plt_rng_Elow_Cs,plt_rng_Ehigh_Cs], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Cs-plt_rng_Tlow_Cs)/(plt_rng_Ehigh_Cs-plt_rng_Elow_Cs))
ax1.axhline(y=Ehist_min_Cs,c="red",linewidth=1.2,linestyle='dashed')
ax1.axhline(y=Ehist_max_Cs,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Cs,plt_rng_Ehigh_Cs])
plt.xlim([plt_rng_Tlow_Cs,plt_rng_Thigh_Cs])
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax2 = fig.add_subplot(322);
ax2.errorbar(Tprof_Cs,Eprof_Cs,yerr=Eerr_Cs)
ax2.plot(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01),prof_Cs(np.arange(plt_rng_Tlow_Cs,plt_rng_Thigh_Cs,0.01)),'-',color='black',linewidth=3)
ax2.plot(xfit_Cs, pfit_Cs(xfit_Cs), '--', color='red', label="m/b = {:.2f} $\\times 10^{{-4}}$ mm$^{{-1}}$".format(m_Cs/b_Cs*1e4))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Cs,Ehist_max_Cs))
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax3 = fig.add_subplot(323);
#ax1.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ecplot[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_D,-plt_rng_Elow_D],[plt_rng_Tlow_D,plt_rng_Thigh_D]])
plt.imshow(h, extent=[plt_rng_Tlow_D,plt_rng_Thigh_D,plt_rng_Elow_D,plt_rng_Ehigh_D], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_D-plt_rng_Tlow_D)/(plt_rng_Ehigh_D-plt_rng_Elow_D))
ax3.axhline(y=Ehist_min_D,c="red",linewidth=1.2,linestyle='dashed')
ax3.axhline(y=Ehist_max_D,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_D,plt_rng_Ehigh_D])
plt.xlim([plt_rng_Tlow_D,plt_rng_Thigh_D])
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax4 = fig.add_subplot(324);
ax4.errorbar(Tprof_D,Eprof_D,yerr=Eerr_D)
ax4.plot(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01),prof_D(np.arange(plt_rng_Tlow_D,plt_rng_Thigh_D,0.01)),'-',color='black',linewidth=3)
ax4.plot(xfit_D, pfit_D(xfit_D), '--', color='red', label="m/b = {:.2f} $\\times 10^{{-4}}$ mm$^{{-1}}$".format(m_D/b_D*1e4))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_D,Ehist_max_D))
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax5 = fig.add_subplot(325);
#ax5.scatter(A_nhits[cuts_T],A_Ec[cuts_T]*CAL_m+CAL_b,s=0.5)
h, x, y = np.histogram2d(-(A_Ecplot[cuts_T]), A_zmax[cuts_T] - A_zmin[cuts_T], bins=[nbins_E, nbins_ltrk], range=[[-plt_rng_Ehigh_Tl,-plt_rng_Elow_Tl],[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl]])
plt.imshow(h, extent=[plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,plt_rng_Elow_Tl,plt_rng_Ehigh_Tl], interpolation = "none", cmap='jet', aspect=(plt_rng_Thigh_Tl-plt_rng_Tlow_Tl)/(plt_rng_Ehigh_Tl-plt_rng_Elow_Tl))
ax5.axhline(y=Ehist_min_Tl,c="red",linewidth=1.2,linestyle='dashed')
ax5.axhline(y=Ehist_max_Tl,c="red",linewidth=1.2,linestyle='dashed')
plt.colorbar()
plt.title("Energy vs. z-length")
plt.ylim([plt_rng_Elow_Tl,plt_rng_Ehigh_Tl])
plt.xlim([plt_rng_Tlow_Tl,plt_rng_Thigh_Tl])
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

ax6 = fig.add_subplot(326);
ax6.errorbar(Tprof_Tl,Eprof_Tl,yerr=Eerr_Tl)
ax6.plot(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01),prof_Tl(np.arange(plt_rng_Tlow_Tl,plt_rng_Thigh_Tl,0.01)),'-',color='black',linewidth=3)
ax6.plot(xfit_Tl, pfit_Tl(xfit_Tl), '--', color='red', label="m/b = {:.2f} $\\times 10^{{-4}}$ mm$^{{-1}}$".format(m_Tl/b_Tl*1e4))
plt.legend(loc=1, fontsize=14)
plt.title("Energy vs. z-length profile: E $\in$ ({0},{1})".format(Ehist_min_Tl,Ehist_max_Tl))
plt.xlabel('z-length (mm)')
plt.ylabel('E (PE)')

plt.savefig("{}/CSTH_E_vs_zlen_PE.{}".format(save_dir,"png"), bbox_inches='tight')